In [99]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Warnings

In [100]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

# Import Necessary Libraries

In [101]:
import numpy as np
import cv2 
import os

from collections import OrderedDict

import torch
from torch import nn
from torchvision.io import read_image,ImageReadMode
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit, StratifiedKFold, KFold
from sklearn import preprocessing
from sklearn.metrics import matthews_corrcoef, accuracy_score, balanced_accuracy_score, precision_score, f1_score, recall_score

from matplotlib import pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.image as mpimg

import json

import pandas as pd

import time
import datetime



# Data Preprocessing

* load training labels.txt file and split the data set into train and valid dataset using simple Stratified or kfold stratified.
* Setup Dataset and Dataloader in Pytorch for loading training, validation, test and noisy dataset and for applying transformations and data augmentation

Transformations (examples):
   * Normalize
   * resize
   * zero centering (AlexNet, VGG etc) during training and depedning the method, save values calculated during training and use them to apply on validation, testing and training
   * denoise the image (if required...how to know that?)
       
Data Augmentation (examples):
   * cropping, scaling images, similar to ResNet,or maybe something else etc (which ones to use will depend upon type of images we expect in the real world...or in our project in noisy test/test dataset)
   * If we know somehow what kind of noise will be there in noisy_test/test dataset (e.g. gaussion noise, salt and pepper, etc) then we can add something similar to training dataset to create more images and use them to train the model as well

Make use of GPU

Save all the hyperparameters somewhere. 

## Split the data

In [102]:
def split_data(y, split_type, test_size=0.3, n_folds=5):
    """
    Split the datasets

    Parameters:
        y: array of labels and image ids with shape (num_images, 2)
        split_type: type of splitting ("simple", "simpleStratified", "kfold", "kfoldStratified")
        test_size: fraction of data for testing (default=0.3)
        n_folds: number of folds (default=5)
    Returns:
        train_labels: numpy array of shape(folds, num_images, 2)
        valid_labels: numpy array of shape(folds, num_images, 2)
    
    """

    if split_type=="simple":
        
        print(f"Splitting data using Simple {100-test_size*100}-{test_size*100} ratio")
       
        # shuffle data before split
        ss = ShuffleSplit(n_splits=1, test_size=test_size)

        for train_index, valid_index in ss.split(X=np.ones(len(y))):
            y_train = np.expand_dims(y[train_index], axis=0)
            y_valid = np.expand_dims(y[valid_index], axis=0)
    
    elif split_type=="simpleStratified":
        
        
        print(f"Splitting data using Simple Stratified with {100-test_size*100}-{test_size*100} ratio")

        # shuffle data before split
        ss = StratifiedShuffleSplit(n_splits=1, test_size=test_size)

        for train_index, valid_index in ss.split(X=np.ones(len(y)), y=y[:,1]):
            y_train = np.expand_dims(y[train_index], axis=0)
            y_valid = np.expand_dims(y[valid_index], axis=0)

    
    elif split_type=="kfold":
        
        print(f"Splitting data using kfolds with number of folds: {n_folds}")

        # shuffle data before creating folds
        kf = KFold(n_splits=n_folds, shuffle=True)
        
        y_train = []
        y_valid = []
        for train_index, valid_index in kf.split(X=np.ones(len(y))):
            y_train.append(y[train_index])
            y_valid.append(y[valid_index])
        
        y_train = np.array(y_train)
        y_valid = np.array(y_valid)

    
    elif split_type == "kfoldStratified":
        
        
        print(f"Splitting data using kfold Stratfication with number of folds: {n_folds}")

        # shuffle data before creating folds
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True)
        y_train = []
        y_valid = []
        
        for train_index, valid_index in skf.split(X=np.ones(len(y)), y=y[:,1]):

            y_train.append(y[train_index])
            y_valid.append(y[valid_index])

        y_train = np.array(y_train)
        y_valid = np.array(y_valid)

    else:
        raise ValueError(f"Unknown value encountered for the parameter 'split_type' during splitting of the data. received {split_type}")

    return y_train, y_valid

In [103]:

#path_to_labels = "/kaggle/input/ismdatasetforclassificationofdieases/code_testing/code_testing/train_binary.txt"
#split_type = "simple"
#test_size= 0.2
#n_folds = 4
#img_ids_labels = np.loadtxt(path_to_labels, dtype=str, delimiter=" ")
#y_train, y_valid = split_data(y=img_ids_labels, split_type=split_type, test_size=test_size, n_folds=n_folds)  
#print("train")
#print(y_train.shape)
#print("valid")
#print(y_valid.shape)


## Encode labels

In [104]:
def label_encoder(y, classes, to_numbers):
    """
    encodes target labels with value between 0 and n_classes-1 and vice versa

    Parameter:
        y: labels of numpy array (num_images,)
        classes: numpy array of names of classes
        to_numbers True/False: whether to transfer from string to numbers or vice versa

    Returns:
        result: encoded labels of numpy array (num_images, 1)
                or labels of numpy array (num_images, 1)
    """

    le = preprocessing.LabelEncoder()
    
    if type(classes).__name__ == 'list':
        classes = np.array(classes)
    le.classes_ = classes
    
    if to_numbers:
        return le.transform(y.ravel())
    else:
        return le.inverse_transform(y.ravel().astype(int)) 


In [105]:
#classes = np.array(["Normal", "COVID", "pneumonia", "Lung_Opacity"]) 
#classes = np.array(["NO_COVID", "COVID"])
#for nf in range(n_folds):
#    y_train[nf, :, 1] = label_encoder(y=y_train[nf,:,1], classes=classes, to_numbers=True)
#    y_valid[nf, :, 1] = label_encoder(y=y_valid[nf,:,1], classes=classes, to_numbers=True)

#print("y-train: ", y_train.shape)
#print("y-valid: ", y_valid.shape)

## Dataset 

In [106]:
class ImageDataset(Dataset):
    """
    
    Parameters:
        labels: labels of shape(num_images, 2) or (num_images, 1)
        path_to_images: path to the folder containing the images
        color: whether to read image in "gray" or "rgb" (default="gray")
        transform: transform to be applied on a sample
        
        When combined with dataloader, it returns:
            image: tensor of shape (batch, 1, height, width)
            label: tensor of shape (batch)
    
    """
    def __init__(self, labels, path_to_images, color="gray", transform=None):
        self.img_labels = labels
        self.img_dir = path_to_images
        self.transform = transform
        
        if color =="gray":
            self.color = ImageReadMode.GRAY
        elif color =="rgb":
            self.color = ImageReadMode.RGB
        else:
            raise ValueError("unknown color passed to dataset")
        

    def __len__(self):
        return self.img_labels.shape[0]

    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.img_labels[index, 0])
        
        # read image 
        image = read_image(img_path, self.color).float()
        
        if self.img_labels.shape[-1]==2:
            label = self.img_labels[index, 1].astype(int)
        else:
            label = -1.0 # just place holder for valid data
            
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [107]:
#path = "/kaggle/input/ismdatasetforclassificationofdieases/code_testing/code_testing/train/0a01d14b-2c8b-4155-ae95-095f625315bd.png"
#image = read_image(path, ImageReadMode.GRAY)
#print(image.size())
#plt.imshow(image[0])

In [108]:
#image2 = read_image(path, ImageReadMode.RGB)
#print(image2.size())
#plt.imshow(image2.permute(1,2,0))

In [109]:
#import PIL as p
#img2 = p.Image.open(path).convert('RGB')
#print(img2.size)
#plt.imshow(img2)

In [110]:
#trans = transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)
#trans_image = trans(image)
#print(trans_image.size())
#plt.imshow(trans_image.permute(1,2,0))

In [111]:
#np.sum(trans_image.numpy() == image2.numpy())

data preprocessing

In [112]:
def data_preprocessing(transformations):
    """
    Returns compose of transforms
    
    Parameters:
        transformations: Ordered dict with structure
        transformations["transform_1"] = {}
        transformations["transform_1"]["name"] = value
        transformations["transform_1"]["parameter_1"] = value
        transformations["transform_1"]["parameter_2"] = value
        
        transformations["transform_2"] = {}
        transformations["transform_2"]["name"] = value
        transformations["transform_2"]["parameter_1"] = value
        transformations["transform_2"]["parameter_2"] = value
        
    Currently supports transformation: 
        "normalize": 
                    keys: mean (default=0) 
                           std (default=255), 
        "resize": 
                    keys: output_shape, 
        
        "random_resized_crop"
                    keys: output_shape
        
        "random_horizontal_flip"
                    keys: p = 0.5
        
        "random_vectical_flip"
                    keys: p = 0.5           
        
        "center_crop"
                    keys: output_shape
                    
        "random_rotation"
                    keys: degrees (defines the range to select from (-deg, +deg)) # in degrees
                        expand(default=False, whether to increase the size of image so that it can fit whole roatated image)
                        
    
    # train transform will be different from valid transform (CHECK)
    """
    
    preprocess = []
    
    for tran in transformations.keys():
        tran_name = transformations[tran]["name"]
        
        # normalization
        if tran_name == "normalize":
            norm = transformations[tran].keys()
            if "mean" in norm:
                m = transformations[tran]["mean"]
                if type(m) == list:
                    mean = m
                else:
                    mean = [m]
            else:
                mean = [0]
            
            if "std" in norm:
                s =transformations[tran]["std"]
                if type(s) == list:
                    std = s
                else:
                    std = [s]
            else:
                std = [255]
                
            preprocess.append(transforms.Normalize(mean=mean,
                             std=std))
        # resize
        elif tran_name == "resize":
            resize = transformations[tran].keys()
            
            if "output_shape" in resize:
                output_shape = transformations[tran]["output_shape"]
            else:
                raise ValueError("Output shape must be provided while resizing")
            
            preprocess.append(transforms.Resize(output_shape))
            
        # random resize crop
        elif tran_name == "random_resized_crop":
            rrc = transformations[tran].keys()
            
            if "output_shape" in rrc:
                output_shape = transformations[tran]["output_shape"]
            else:
                raise ValueError("Output shape must be provided while random_resized_crop") 
            
            preprocess.append(transforms.RandomResizedCrop(output_shape))
        
        # random horizontal flip
        elif tran_name == "random_horizontal_flip":
            rhf = transformations[tran].keys()
            
            if "p" in rhf:
                p = transformations[tran]["p"]
            else:
                p = 0.5
            
            preprocess.append(transforms.RandomHorizontalFlip(p))
         
        # random vertical flip
        elif tran_name == "random_vertical_flip":
            rvf = transformations[tran].keys()
            
            if "p" in rvf:
                p = transformations[tran]["p"]
            else:
                p = 0.5
            
            preprocess.append(transforms.RandomVerticalFlip(p))
            
        # center crop
        elif tran_name == "center_crop":
            cc = transformations[tran].keys()
            
            if "output_shape" in cc:
                output_shape = transformations[tran]["output_shape"]
            else:
                raise ValueError("Output shape must be provided while center_crop") 
            
            preprocess.append(transforms.CenterCrop(output_shape))
        
        # random_rotation
        elif tran_name == "random_rotation":
            rr = transformations[tran].keys()
            
            if "degrees" in rr:
                degrees = transformations[tran]["degrees"]
            else:
                raise ValueError("degrees must be provided while random rotation")
            
            if "expand" in rr:
                expand = transformations[tran]["expand"]
            else:
                expand = False
            
            preprocess.append(transforms.RandomRotation(degrees=degrees, expand=expand))
        
        else:
            raise ValueError("Unknown transformation passed")
    
    return transforms.Compose(preprocess)

In [113]:

#path_to_images = "/kaggle/input/ismdatasetforclassificationofdieases/code_testing/code_testing/train"
#batch_size = 2

#data_transform = OrderedDict()
#data_transform["t1"] = {}
#data_transform["t1"]["name"] = "normalize"
#data_transform["t1"]["mean"] = 0
#data_transform["t1"]["std"] = 255
#data_transform["t2"] = {}
#data_transform["t2"]["name"] = "resize"
#data_transform["t2"]["output_shape"] = (5,5)


#data_transforms = data_preprocessing(data_transform)

#data_transforms = transforms.Compose([
#        transforms.RandomSizedCrop(224),
#        transforms.RandomHorizontalFlip(),
#        transforms.ToTensor(),
#        transforms.Normalize(mean=[0],
#                             std=[255]),
#        transforms.Resize((250,250))
#    ])



#for fold in range(n_folds):

    #train_data = ImageDataset(y_train[fold], path_to_images, transform=data_transforms)
    #train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
#    print(len(train_dataloader))
 #   for x, y in train_dataloader:"""
  #  #print(f"Fold No: {fold} Train: x shape: {x.shape} y_shape: {y.shape}")
   # """
   # valid_data = ImageDataset(y_valid[fold], path_to_images, transform=data_transforms)
   # valid_dataloader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
  #  print(len(valid_dataloader))
  #  for x, y in valid_dataloader:"""
#print(f"Fold No: {fold} Valid: x shape: {x.shape} y_shape: {y.shape}")


# Setup Model

Model setup could be done using the following ways:
   1. Model with user defined layers.
   2. Pretrained Model
   3. Ensemble of models

Layers could be:
   * Convolution
   * Activation (Linear, ReLU, LReLU, PReLU, tanh, etc)
   * Pooling (Max, Average, Global)
   * Fully Connected Layer
   * Dropout
   * Batch Normalization
and many others

These layers could be added to the any model. Which layers to be used can be find by hit and trial method or from literature

Pretrained Model could be:
   * VGG
   * ResNet
   * GoogleLeNet
and many others.

Which pretrained model to use can be find by literature and how many layers to be trained can be determined by hit and trial or from literature.

We can always combine pretrained model with own custom model.

Make use of GPU

Save the model, and all the hyperparamters and all the info that may be learned during training and may be required in testing phase

In [114]:
class Model(nn.Module):
    """
    Sets up the model
    layers: OrderedDict with format:
        layers["layer_1"] = {}
        layers["layer_1"]["name"] = value
        layers["layer_1"]["parameter1"] = value
        layers["layer_1"]["parameter2"] = value
        
        layers["layer_2"] = {}
        layers["layer_2"]["name"] = value
        layers["layer_2"]["parameter1"] = value
        layers["layer_2"]["parameter2"] = value
        
    currently supports layer names. "linear", "flatten", "relu", "lrelu"
    
    output layer must be added as well
    """
    def __init__(self, input_shape, layers, use_pretrained_model):
        super(Model, self).__init__()
        self.input_shape = input_shape
        self.layers = layers
        if use_pretrained_model:
            self.seq_model = self.get_pretrained_model()
        else:
            self.seq_model = nn.Sequential(self.get_model())
    
    def forward(self, x):
        return self.seq_model(x)
    
    def get_model(self):
        
        arch = []
        self.layer = "Init"
        self.pre_layer_output = self.input_shape  
        
        for layer in self.layers.keys():
            self.layer = layer
            layer_name = self.layers[layer]["name"] 
            
            # Linear Layer 
            if layer_name == "linear":
                arch.append(self.add_linear(self.layers[layer]))
                
            # ReLU
            elif layer_name == "relu":
                arch.append(self.add_relu(self.layers[layer]))
                
            # LeakyReLU
            elif layer_name == "lrelu":
                arch.append(self.add_lrelu(self.layers[layer]))
            
            # flatten
            elif layer_name == "flatten":
                arch.append(self.add_flatten(self.layers[layer]))
                
            # conv
            elif layer_name == "conv":
                arch.append(self.add_conv(self.layers[layer]))
            
            # max pooling
            elif layer_name == "max_pool":
                arch.append(self.add_max_pool(self.layers[layer]))
            
            else:
                raise ValueError("Unknown layer encountered")

        return OrderedDict(arch)
    
    def get_pretrained_model(self):
        
        if len(self.layers.keys()) != 1:
            raise ValueError("Currently only single pretrained model is supported")
        
        for layer in self.layers.keys():
            self.layer = layer
            layer_name = self.layers[layer]["name"] 

            # mobile_net 
            if layer_name == "mobile_net":
                return self.add_mobile_net(self.layers[layer])

            # resnet18
            elif layer_name == "resnet":
                return self.add_resnet(self.layers[layer])
            
            # resnet50
            elif layer_name == "resnet50":
                return self.add_resnet50(self.layers[layer])

            else:
                raise ValueError("Unknown pretrained model name encountered")
    
    def set_parameter_requires_grad(self, features, flag):
        """
        Sets requires grad of layers to True/False
        """
        for param in features.parameters():
            param.requires_grad = flag

    def add_linear(self, parameters):
        """
        Adds Linear Layer
        
        parameters: dict containing keys:
            neurons: number of neurons in layer
            bias: whether to add bias term (default = True)
        """
        param = parameters.keys()
        
        if "neurons" in param:
            neurons = parameters["neurons"]
        else:
            raise ValueError ("Number of neurons must be provided")
            
        if "bias" in param:
            bias = parameters["bias"]
        else:
            bias = True
            
        linear = (f'{self.layer}', nn.Linear(
                                    in_features = self.pre_layer_output, 
                                    out_features= neurons, 
                                    bias = bias))
        self.pre_layer_output = neurons
        
        return linear
    
    def add_relu(self, parameters):
        """
        Adds ReLU layer
        """
        return (f'{self.layer}', nn.ReLU())
    
    def add_lrelu(self, parameters):
        """
        Adds ReLU layer
        Parameters:
            alpha: negative slope (default = 1e-2)
        """
        if "alpha" in parameters.keys():
            alpha = parameters["alpha"]
        else:
            alpha = 1e-2
        return (f'{self.layer}', nn.LeakyReLU(alpha))
    
    def add_flatten(self, parameters):
        """
        Adds Flatten layer
        """
        flatten_layer = (f'{self.layer}', nn.Flatten())
        in_shape = self.pre_layer_output
        
        # calculate output shape
        out_shape = 1
        for d in range(len(in_shape)):
            out_shape *= in_shape[d]
        
        self.pre_layer_output = out_shape
        
        return flatten_layer
    
    def add_conv(self, parameters):
        """
        Adds Conv layer
        
        Parameters:
            number_of_kernels: int
            kernel_size: int or tuple of ints
            stride: int or tuple of ints (default=1)
            padding: 'valid' or 'same' or tuple of ints (default='valid')
            bias: default=True
            
        """
        param_keys = parameters.keys()
        
        if "number_of_kernels" in param_keys:
            out_channels = parameters["number_of_kernels"]
        else:
            raise ValueError("When adding Convulational Layer, kernel_size must be provided")
        
        
        if "kernel_size" in param_keys:
            kernel_size = parameters["kernel_size"]
        else:
            raise ValueError("When adding Convulational Layer, kernel_size must be provided")
        
        if "stride" in param_keys:
            stride = parameters["stride"]
        else:
            stride = 1
            
        if "padding" in param_keys:
            padding = parameters["padding"]
        else:
            padding = 'valid'
        
        if "bias" in param_keys:
            bias = parameters["bias"]
        else:
            bias = True
        
        if len(self.pre_layer_output) == 2:
            in_channels = 1
            in_height, in_width =  self.pre_layer_output
        else:
            in_channels, in_height, in_width =  self.pre_layer_output
        
        if padding == 'valid':
            pad = 0
        
        if padding == 'same':
            pad =  ( (in_height - 1)* stride + kernel_size - in_height)/2
       
        
        out_height = (in_height + 2*pad - kernel_size)/ stride + 1
        out_width = (in_width + 2*pad - kernel_size)/ stride + 1
        
        if out_width %1 !=0 or out_height %1 !=0 :
            raise ValueError(f"Combination of pad, stride and kernel size lead to decimal number in conv layer. {out_height}")

        self.pre_layer_output = (out_channels, int(out_height), int(out_width))
        
        conv_layer = (f'{self.layer}', nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias))
        
        return conv_layer
    
    def add_max_pool(self, parameters):
        """
        Adds Max pooling layer
        
        Parameters:
            kernel_size: int or tuple of ints
            stride: int or tuple of ints (default=2)
            padding:int or tuple of ints (default=0)
        
        """
        
        param_keys = parameters.keys()
        
        if "kernel_size" in param_keys:
            kernel_size = parameters["kernel_size"]
        else:
            raise ValueError("When adding Convulational Layer, kernel_size must be provided")
        
        if "stride" in param_keys:
            stride = parameters["stride"]
        else:
            stride = 2
            
        if "padding" in param_keys:
            padding = parameters["padding"]
        else:
            padding = 0
        
        
        if(len(self.pre_layer_output)) != 3:
            raise ValueError(f"The input shape to the Max Pooling Layer is not correct. {self.pre_layer_output}")
         
        in_channels, in_height, in_width =  self.pre_layer_output
        
        out_height = (in_height + 2*padding - kernel_size)/ stride + 1
        out_width = (in_width + 2*padding - kernel_size)/ stride + 1
        out_channels = in_channels
       
        if out_width %1 !=0 or out_height %1 !=0 :
            raise ValueError(f"Combination of pad, stride and kernel size lead to decimal number in conv layer. {out_height}")


        self.pre_layer_output = (out_channels, int(out_height), int(out_width))
        
        max_pool_layer = (f'{self.layer}', nn.MaxPool2d(kernel_size=kernel_size, stride=stride, padding=padding))
        
        return max_pool_layer
    
    def add_mobile_net(self, parameters):
        """
        Returns a mobile net pretrained model. Currently supports V2 weights
        
        Parameters:
            weights: default = 'IMAGENET1K_V2'
            num_output_neurons = number of neurons in the last layer
            num_layers_to_train = number of layers to train. it starts counting from last layer. (default=0 -> only last layer is trained)
                                pretrained weights will be used for layers to be trained.
                                By passing -1, all layers will be trained. this is equal to finetuning.
                                
        """
        param_keys= parameters.keys()
        
        if "weights" in param_keys:
            weights = parameters["weights"]
        else:
            weights = 'IMAGENET1K_V2'
        
        if "num_output_neurons" in param_keys:
            num_output_neurons = parameters["num_output_neurons"]
        else:
            raise ValueError("num_output_neurons must be provided while using mobile net")
            
        if "num_layers_to_train" in param_keys:
            num_layers_to_train = parameters["num_layers_to_train"]
        else:
            num_layers_to_train = 0
        
        mobile_net = models.mobilenet_v2(pretrained=True)
        
        if num_layers_to_train >=0:
            self.set_parameter_requires_grad(mobile_net, False)
            
            if num_layers_to_train > 0:
                train_features_index = len(mobile_net.features) - num_layers_to_train
                self.set_parameter_requires_grad(mobile_net.features[train_features_index:], True)
                

        in_features = mobile_net.classifier[-1].in_features
        mobile_net.classifier[-1] = nn.Linear(in_features, num_output_neurons)
                
        return mobile_net
        
        
    def add_resnet(self, parameters):
        """
        Returns resnet pretrained model. Currently supports ResNet18 and supports IMAGENET1K_V1
        
        Parameters:
            weights: default = 'IMAGENET1K_V1'
            num_output_neurons = number of neurons in the last layer
            num_layers_to_train = number of layers to train. it starts counting from last layer. (default=0 -> only last layer is trained)
                                pretrained weights will be used for layers to be trained.
                                By passing -1, all layers will be trained. this is equal to finetuning.
                                
        """
        param_keys= parameters.keys()
        
        if "weights" in param_keys:
            weights = parameters["weights"]
        else:
            weights = 'IMAGENET1K_V1'
        
        if "num_output_neurons" in param_keys:
            num_output_neurons = parameters["num_output_neurons"]
        else:
            raise ValueError("num_output_neurons must be provided while using mobile net")
            
        if "num_layers_to_train" in param_keys:
            num_layers_to_train = parameters["num_layers_to_train"]
        else:
            num_layers_to_train = 0
            
        resnet = models.resnet18(pretrained=True)
        
        if num_layers_to_train >=0:
            self.set_parameter_requires_grad(resnet, False)
            
            if num_layers_to_train > 0:
                print("Training previous layers in ResNet18 is not supported yet")
                
                #train_features_index = len(resnet.features) - num_layers_to_train
                #self.set_parameter_requires_grad(resnet.features[train_features_index:], True)
                

        in_features = resnet.fc.in_features
        resnet.fc = nn.Linear(in_features, num_output_neurons)
                
        return resnet
        
    def add_resnet50(self, parameters):
        """
        Returns resnet pretrained model. Currently supports ResNet18 and supports IMAGENET1K_V1
        
        Parameters:
            weights: default = 'IMAGENET1K_V1'
            num_output_neurons = number of neurons in the last layer
            num_layers_to_train = number of layers to train. it starts counting from last layer. (default=0 -> only last layer is trained)
                                pretrained weights will be used for layers to be trained.
                                By passing -1, all layers will be trained. this is equal to finetuning.
                                
        """
        param_keys= parameters.keys()
        
        if "weights" in param_keys:
            weights = parameters["weights"]
        else:
            weights = 'IMAGENET1K_V1'
        
        if "num_output_neurons" in param_keys:
            num_output_neurons = parameters["num_output_neurons"]
        else:
            raise ValueError("num_output_neurons must be provided while using mobile net")
            
        if "num_layers_to_train" in param_keys:
            num_layers_to_train = parameters["num_layers_to_train"]
        else:
            num_layers_to_train = 0
            
        resnet = models.resnet50(pretrained=True)
        
        if num_layers_to_train >=0:
            self.set_parameter_requires_grad(resnet, False)
            
            if num_layers_to_train > 0:
                print("Training previous layers in ResNet50 is not supported yet")
                
                #train_features_index = len(resnet.features) - num_layers_to_train
                #self.set_parameter_requires_grad(resnet.features[train_features_index:], True)
                

        in_features = resnet.fc.in_features
        resnet.fc = nn.Linear(in_features, num_output_neurons)
                
        return resnet

In [115]:
# order is important
#layers = OrderedDict()

#layers["pretrained_model"] = {}
#layers["pretrained_model"]["mobile_net"] = {}
#layers["pretrained_model"]["mobile_net"]["name"] = "mobile_net"
#layers["pretrained_model"]["mobile_net"]["num_output_neurons"] = 4
#layers["pretrained_model"]["mobile_net"]["num_layers_to_train"] = 5


#model = Model((5,5), layers["pretrained_model"], True )

#print(model)

In [116]:
#for name,param in model.named_parameters():
    
    #param.requires_grad = False
    #print(f"name {name} param {param.requires_grad}")

# Setup loss function
* Binary entropy loss 
* cross entropy loss
* weighted loss function
* maybe something from literature

In [117]:
def setup_loss(parameters):
    """
    Returns loss function
    
    Parameters:
        parameters: dict with the following keys:
            "type": "binary" or "cross_entropy"
            "use_weighted_loss": default = False
            "class_weights": weights for each class (list)
        
        Note: currently the loss function is basically Softmax + Log Loss.
        loss_type == "binary" assumes that output layer has single neuron
        When loss_type == "binary", class weights must be equal to [num_of_neg_samples/num_of_pos_samples]
        else: class_weights can be list containing any value but length of class_weights must be equal to number of classes
        
    """
    
    loss_keys = parameters.keys()
    
    if "type" in loss_keys:
        loss_type = parameters["type"]
    else:
        raise ValueError ("'loss_type' must be provided")
         
    if "use_weighted_loss" in loss_keys:
        use_weighted_loss = parameters["use_weighted_loss"]
    else:
        use_weighted_loss = False
    
    if "class_weights" in loss_keys:
        class_weights = parameters["class_weights"]
    else:
        class_weights = []

    if "device" in loss_keys:
        device = parameters["device"]
    else:
        device = "cpu"


        
    if use_weighted_loss and not class_weights:
        raise ValueError("When using weighted loss, class weights must be provided")

    class_weights = torch.tensor(class_weights)
    if device != "cpu":    
        class_weights = torch.tensor(class_weights).cuda()
    
    if loss_type == "binary":
        
        if use_weighted_loss:
            print("BC: using Weighted Loss")
            loss_fnt = nn.BCEWithLogitsLoss(pos_weight=class_weights)
        else:
            loss_fnt = nn.BCEWithLogitsLoss()
    
    elif loss_type == "cross_entropy":
         
        if use_weighted_loss:
            print("CE: using Weighted Loss")
            loss_fnt= nn.CrossEntropyLoss(weight=class_weights)
        else:

            loss_fnt= nn.CrossEntropyLoss()
            
    else:
        raise ValueError("Unknown Value for loss_type")
    
    return loss_fnt
    

In [118]:
#loss_param = {}
#loss_param["type"] = "cross_entropy"
#loss_param["use_weighted_loss"] = True
#loss_param["class_weights"] = [8, 6]

#loss_fnt = setup_loss(loss_param)
#print(loss_fnt)

# Setup Optimizer
* adam
* SGD with momenteum
* something from literature
* regularization
* learning rate change with time (if required) 
* betas

In [119]:
def setup_optimizer(model_parameters, optim_parameters):
    """
    Setups Optimizer
    
    Parameters:
        model_parameters: parameters of models that will be trained
        
        optim_parameters: dict with the following structure:
            optim_parameters["name"] = name of optimizer (adam or sgd)
            optim_parameters["parameter1"] = value
            optim_parameters["parameter2"] = value
            
        for Adam optimizer, it supports following keys:
            "betas": default = (0.9,0.999)
            
        for SGD, it supports following keys.
            "momentum":  (default = 0)
            "dampening": (default=0)
        
        following keys are supported for both optimizers:
            "lr": learning rate (default = 1e-3)
            "lmbda": regularization (default = 0)     
            
    """
    
    optim_keys=optim_parameters.keys()
    
    if "name" in optim_keys:
        optim = optim_parameters["name"]
    else:
        raise ValueError("'name' of the optimizer must be provided")
    
    if "lmbda" in optim_keys:
        lmbda = optim_parameters["lmbda"]
    else:
        lmbda = 0
    
    if "lr" in optim_keys:
        lr = optim_parameters["lr"]
    else: 
        lr = 1e-3
    
    if optim == "adam":
        
        if "betas" in optim_keys:
            betas = optim_parameters["betas"]
        else:
            betas = (0.9,0.999)
        
        optimizer = torch.optim.Adam(model_parameters,  lr=lr, betas=betas, amsgrad=True, weight_decay=lmbda)
    
    elif optim == "sgd":
        
        if "momentum" in optim_keys:
            momentum = optim_parameters["momentum"]
        else:
            momentum = 0
        
        if "dampening" in optim_keys:
            dampening = optim_parameters["dampening"]
        else:
            dampening = 0
        
        optimizer = torch.optim.SGD(model_parameters, momentum=momentum, dampening=dampening, weight_decay=lmbda)
    else:
        raise ValueError("Unknown name of the optimizer provided")
    
    return optimizer

In [120]:
#optim_parameters = {}
#optim_parameters["name"] = "adam"
#optimizer = setup_optimizer (model.parameters(), optim_parameters)
#print(optimizer)

# Metrics

## Accuracy 

In [121]:
def accuracy(y_true, y_pred, parameters):
    """
    Calculates accuracy
    
    Parameters:
        y_true: numpy array of shape (num_images,)
        y_pred: numpy array of shape (num_images,)
        parameters: dictionary with the following keys:
            type: "simple" (default) or "balanced"
    
    Returns:
        score: float
        config:

    Additional Notes:

        Simple Accuracy: fraction of labels that are equal.
        Balanced Accuracy: 
            Binary class: equal to the arithmetic mean of sensitivity (true positive rate) 
                            and specificity (true negative rate)
                             = 1/2 ( (TP/TP+FN) + (TN/TN+FP))

            Multiclass: the macro-average of recall scores per class
                        recall for each class and then take the mean
                        recall = TP /(TP+FN)

            if the classifier predicts same label for all examples, the score will be equal to 1/num_classes (for binary: 0.5)
            for more info, see
                "https://scikit-learn.org/stable/modules/model_evaluation.html#balanced-accuracy-score"
    """

    config={}
    if "type" in parameters.keys():
        accu_type = parameters["type"]
    else:
        accu_type = "simple"

    config["type"] = accu_type
    
    if accu_type == "simple":
        score = accuracy_score(y_true=y_true, y_pred=y_pred) 
    elif accu_type == "balanced":
        score = balanced_accuracy_score(y_true=y_true, y_pred=y_pred)
    else:
        raise ValueError("Unknown Value encountered for parameter 'type' while calculating accuracy")#

    
    return score, config

In [122]:
#y_true = np.array([0, 2, 1, 3])
#y_predict = np.array([0, 1, 2, 3])
#print("simple: ", accuracy(y_true, y_predict, {}))
#y_true = np.array([0, 1, 0, 0, 1, 0])
#y_predict = np.array([0, 1, 0, 0, 0, 1])
#print("balanced: ", accuracy(y_true, y_predict, {"type": "balanced"}))

## MCC 

In [123]:
def mcc(y_true, y_pred, parameters):
    """
    Calculates mcc
    
    Parameters:
        y_true: numpy array of shape (num_images,)
        y_pred: numpy array of shape (num_images,)
        parameters: dictionary with the following keys:
            
    Returns:
        score: float
        config:

    Additional Notes:
        Binary:
        
            +1 -> prefect, 0-> random, -1 -> inverse 

            mcc = (tp*tn) - (fp*fn) / sqrt( (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  )

        Multiclass:
            
            +1 -> perfect, between -1 and 0 -> min

            for more info, see
                "https://scikit-learn.org/stable/modules/model_evaluation.html#matthews-corrcoef"
    """

    config = {}

    score = matthews_corrcoef(y_true=y_true, y_pred=y_pred)

    return score, config

In [124]:
#y_true = np.array([+1, +1, +1, -1])
#y_predict = np.array([+1, -1, +1, +1])
#print("mcc: ", mcc(y_true, y_predict, {}))

## Sensitivity 

In [125]:
def sensitivity(y_true, y_pred, parameters):
    """
    Calculates sensitivity (recall)
    
    Parameters:
        y_true: numpy array of shape (num_images,)
        y_pred: numpy array of shape (num_images,)
        parameters: dictionary with the following keys:
            class_result: for binary classes, name of class for which metric will be calculated
            average: for mulitlcass, how to calculate metrics for each class: 'micro', 'macro', 'weighted' 
    
    Returns:
        score: float
        config:

    Additional Notes:
        recall = TP/ TP+FN

        average:
            'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives
            'macro': Calculate metrics for each label, and find their unweighted mean
            'weighted': Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label)

    """

    config = {}

    if len(np.unique(y_true)) <= 2 and len(np.unique(y_pred)) <= 2:

        if "class_result" in parameters.keys():
            pos_label = parameters["class_result"]
        else:
           ytl = np.unique(y_true)
           ypl = np.unique(y_pred)
           labels = np.unique(np.concatenate((ytl, ypl))) 
           pos_label = labels[0]
        
        config["class_result"] = pos_label

        score = recall_score(y_true=y_true, y_pred=y_pred, pos_label=pos_label)
    
    else:
        
        if "average" in parameters.keys():
            average = parameters["average"]
        else:
            average = "weighted"

        config["average"] = average

        score = recall_score(y_true=y_true, y_pred=y_pred, average=average)

    return score, config

## Precision 

In [126]:
def precision(y_true, y_pred, parameters):
    """
    Calculates precision
    
    Parameters:
        y_true: numpy array of shape (num_images,)
        y_pred: numpy array of shape (num_images,)
        parameters: dictionary with the following keys:
            class_result: for binary classes, name of class for which metric will be calculated
            average: for mulitlcass, how to calculate metrics for each class: 'micro', 'macro', 'weighted' 
    
    Returns:
        score: float
        config:

    Additional Notes:
        precision = TP/ TP+FP

        average:
            'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives
            'macro': Calculate metrics for each label, and find their unweighted mean
            'weighted': Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label)

    """
    config = {}

    if len(np.unique(y_true)) <= 2 and len(np.unique(y_pred)) <= 2:

        if "class_result" in parameters.keys():
            pos_label = parameters["class_result"]
        else:
           ytl = np.unique(y_true)
           ypl = np.unique(y_pred)
           labels = np.unique(np.concatenate((ytl, ypl))) 
           pos_label = labels[0]
        
        config["class_result"] = pos_label

        score = precision_score(y_true=y_true, y_pred=y_pred, pos_label=pos_label)
    
    else:
        
        if "average" in parameters.keys():
            average = parameters["average"]
        else:
            average = "weighted"

        config["average"] = average

        score = precision_score(y_true=y_true, y_pred=y_pred, average=average)

    return score, config

## F1 Score

In [127]:
def F1_score(y_true, y_pred, parameters):
    """
    Calculates f1 score
    
    Parameters:
        y_true: numpy array of shape (num_images,)
        y_pred: numpy array of shape (num_images,)
        parameters: dictionary with the following keys:
            class_result: for binary classes, name of class for which metric will be calculated
            average: for mulitlcass, how to calculate metrics for each class: 'micro', 'macro', 'weighted' 
    
    Returns:
        score: float
        config:

    Additional Notes:
        f1_score = 2 * (Precision * Recall)/ (Precision + Recall)

        average:
            'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives
            'macro': Calculate metrics for each label, and find their unweighted mean
            'weighted': Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label)

    """

    config = {}

    if len(np.unique(y_true)) <= 2 and len(np.unique(y_pred)) <= 2:

        if "class_result" in parameters.keys():
            pos_label = parameters["class_result"]
        else:
           ytl = np.unique(y_true)
           ypl = np.unique(y_pred)
           labels = np.unique(np.concatenate((ytl, ypl))) 
           pos_label = labels[0]
        
        config["class_result"] = pos_label

        score = f1_score(y_true=y_true, y_pred=y_pred, pos_label=pos_label)
    
    else:
        
        if "average" in parameters.keys():
            average = parameters["average"]
        else:
            average = "weighted"

        config["average"] = average

        score = f1_score(y_true=y_true, y_pred=y_pred, average=average)

    return score, config


## Evaluate Metrics 

In [128]:
def evaluate_metrics(y_true, y_pred, metrics, y_pred_probs=None):
    """
    Applies each metric and generates evaluation score

    Parameters:
        y_true: numpy array of shape (folds, num_images, 2)
        y_pred: numpy array of shape (folds, num_images, 2)
        metrics: dictionary with following structure:
            metrics["metrics_1"]["name"] = name of metric
            metrics["metrics_1"]["parameter_1"] = value
            metrics["metrics_1"]["parameter_2"] = value

            metrics["metrics_2"]["name"] = name of metric
            metrics["metrics_2"]["parameter_1"] = value
            metrics["metrics_2"]["parameter_2"] = value
            
    
    currently, supports "accuracy", "mcc", "precision", "sensitivity", "F1_score"
  

    Returns:
        scores:numpy array of shape (folds, metrics)
        output_config:
       
    Additional Notes:

    """
    # check whether correct shapes of y_* are provided or not

    if len(y_true.shape) !=3:
        raise ValueError("Shape of y_true is not correct.")

    if len(y_pred.shape) == 3:
        num_folds = y_pred.shape[0]
        met_keys = list(metrics.keys())
        num_metrics = len(met_keys)
    else:
        raise ValueError("Shape of y_pred is not correct.")

    config={}

    list_of_metrics = []

    scores = np.full((num_folds, num_metrics), 1000, dtype=np.float32)
    
    flag = True


    for fold_no in range(num_folds):

        for metric_no in range(num_metrics):

            print(f"Processing: Fold No: {fold_no} Metric: {met_keys[metric_no]}")

            metric_name = metrics[met_keys[metric_no]]["name"]
            
            if metric_name == "accuracy":
                fnt_pointer = accuracy
            elif metric_name == "mcc":
                fnt_pointer = mcc
            elif metric_name == "precision":
                fnt_pointer = precision
            elif metric_name =="sensitivity":
                fnt_pointer = sensitivity
            elif metric_name =="F1_score":
                fnt_pointer = F1_score
            else:
                raise ValueError("Unknown metric found")

            
            metric_score, fnt_config = fnt_pointer(y_true=y_true[fold_no, :, 1], 
                                                    y_pred=y_pred[fold_no, :, 1], 
                                                    parameters=metrics[met_keys[metric_no]])


            scores[fold_no, metric_no] = metric_score

            # setup output config
            if flag:
                fnt_config["name"] = metric_name
                config[met_keys[metric_no]] = {} 
                config[met_keys[metric_no]] = fnt_config

                list_of_metrics.append(met_keys[metric_no])

        flag=False
    
    return scores, config, list_of_metrics


In [129]:
# multi
#y_true = np.array(["aa", "a", "ab", "b", "ac", "c",
#                    "ad", "a", "ae", "c", "af", "a",
#                    "ag", "c", "ah", "b", "ai", "a"]).reshape(3,3, 2)
#y_pred = np.array(["aa", "b", "ab", "b", "ac", "b",
#                    "ad", "a", "ae", "c", "af", "a",
#                    "ag", "a", "ah", "b", "ai", "c",]).reshape(3,3,2)




#metrics = {}

#metrics["simple_accuracy"] = {}
#metrics["simple_accuracy"]["name"] = "accuracy"
#metrics["simple_accuracy"]["type"] = "simple"  

#metrics["balanced_accuracy"] = {}
#metrics["balanced_accuracy"]["name"] = "accuracy"
#metrics["balanced_accuracy"]["type"] = "balanced"


# precision
#metrics["precision"] = {}
#metrics["precision"]["name"] = "precision" 
#metrics["precision"]["class_result"] = "a"
#metrics["precision"]["average"] = "weighted"


# recall
#metrics["sensitivity"] = {}
#metrics["sensitivity"]["name"] = "sensitivity"
#metrics["sensitivity"]["class_result"]  = "a"
#metrics["sensitivity"]["average"]  = "weighted"

# f1_score
#metrics["f1_score"] = {}
#metrics["f1_score"]["name"] = "F1_score" 
#metrics["f1_score"]["class_result"] = "a"
#metrics["f1_score"]["average"] = "weighted"

# mcc
#metrics["mcc"] = {}
#metrics["mcc"]["name"] = "mcc" 


#print("true labels")
#print(y_true)
#print(y_true.shape)

#print("pred labels")
#print(y_pred)
#print(y_pred.shape)

#scores, config, list_of_metrics =evaluate_metrics(y_true=y_true, y_pred=y_pred, metrics =metrics)

#print("scores ")
#print(scores)

#print("config ")
#print(config)

#print("metrics list ")
#print(list_of_metrics)

# Plots

## Learning Curves 

In [130]:
def plot_LC(metric_score, path_to_results, path_to_images):

    """
    Plots and saves learning curves
    
    metric_score: numpy array of shape (metrics, 2, epochs)
    path_to_results: path where plot will be saved
    
    """
    num_metrics = metric_score.shape[0]
    metrics = ["CELoss", "Bal_Accu", "MCC"]
    
    fig, axes = plt.subplots(num_metrics, 1)
    
    fig.suptitle("Learning Curves")

    for met in range(num_metrics):

        # plot training curve
        train_line, = axes[met].plot(metric_score[met][0], color='blue', label='Training')

        # plot validation curve
        valid_line, = axes[met].plot(metric_score[met][1], color= 'orangered', label='Validation')

        # setup x-axis
        if met != num_metrics -1 :
            axes[met].set_xticks([])
        else:
            axes[met].set_xlabel("Epochs")

        # setup y-axis
        axes[met].set_ylabel(metrics[met])

    fig.legend(handles= [train_line, valid_line])

    plt.savefig(path_to_results + "_LC")
    plt.close()

In [131]:
#x= np.random.rand(3, 2, 10)
#x[0,0] = np.arange(1, 11)
#x[1, 1] = np.arange(11,21)

#print(x)
#path_to_results = "/"

#plot_LC(
#    x, path_to_results, x)

## Misidentified Samples 

In [132]:
def plot_MS(y_true, y_pred, path_to_results, path_to_images):
    """
    Plots and save missclassified samples
    
    y_true: numpy array of shape (num_of_images,2)
    y_pred: numpy array of shape (num_of_images,2)
    path_to_results: path where plot will be saved
    path_to_images: folder containing images


    Only works with (num_samples)^2 = whole number

    """
    
    num_samples_to_plot = 4

    classes = np.unique(y_true[:,1])

    if np.sqrt(num_samples_to_plot) %1 !=0:
        raise ValueError("Currently, this function only supports those number of samples whose sqaure is a whole number")

    
    
    cm = create_dict(classes)

    for true_label in classes:
        
        # get all images for a class in y_true
        pos = y_true[np.where(y_true[:,1] == true_label)]
        
        pred_classes = classes.tolist()
      
        for sample in pos:
            
            # find given image in y_pred
            img_id = y_pred[np.where(y_pred[:,0] == sample[0])]
            
            # store img_ids in their respective col
            for pred_label in pred_classes:
                if img_id[0,1] == pred_label:
                    cm[true_label][pred_label].append(img_id[0,0])

            # check whether there are requried number of samples in each col
            for l in cm[true_label].keys():
                
                if l in pred_classes and len(cm[true_label][l]) == num_samples_to_plot:
                    pred_classes.remove(l)
                
            if not len(pred_classes):
                break
    
    plot_CM_images(cm, num_samples_to_plot, path_to_images, path_to_results)



def plot_CM_images(cm, num_samples, path_to_images, path_to_results):
    """
    Plots and saves images
    
    """
    num_classes = len(cm.keys())
    
    num_imgs_axis = int(np.sqrt(num_samples)) 
    
    num_rows= num_imgs_axis * num_classes
    num_cols = num_imgs_axis * num_classes

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10,10))
    
    fig.suptitle("Confusion Matrix of misclassified images")
    plt.subplots_adjust(wspace=0, hspace=0)

    fig.text(0.5, 0.04, 'Predicted Labels', ha='center', va='center')
    fig.text(0.06, 0.5, 'True Labels', ha='center', va='center', rotation='vertical')

    for i, true_label in enumerate(cm.keys()):
        
        pred_labels = cm[true_label]
        
        if i==0:
            row_i = i
        

        for j, pred_label in enumerate(pred_labels.keys()):

            img_ids = pred_labels[pred_label]

            empty_img_ids = num_samples - len(img_ids)

            if j==0:
                col_j = j
            
            row = 0
            col = 0

            for id in img_ids:

                img = mpimg.imread(os.path.join(path_to_images, id))
                img_shape = img.shape
                
                axes[row_i+row, col_j+col].imshow(img, cmap='gray', vmin=0, vmax=255, aspect='auto')
                axes[row_i+row, col_j+col].set_xticks([])
                axes[row_i+row, col_j+col].set_yticks([])

                if row_i + row == num_rows-1:
                    axes[row_i+row, col_j+col].set_xlabel(pred_label)

            
                if col_j + col == 0:
                    axes[row_i+row, col_j+col].set_ylabel(true_label)




                if col ==num_imgs_axis-1:
                    col =0
                    row +=1
                else:
                    col +=1

            if len(img_ids) ==0:
                img_shape = (299,299)        
            
            if empty_img_ids > 0:
                temp = np.full(img_shape, 255)

                for _ in range(empty_img_ids):
                    
                    axes[row_i+row, col_j+col].imshow(temp, cmap='gray', vmin=0, vmax=255, aspect='auto')
                    axes[row_i+row, col_j+col].set_xticks([])
                    axes[row_i+row, col_j+col].set_yticks([])

                    if row_i + row == num_rows-1:
                        axes[row_i+row, col_j+col].set_xlabel(pred_label)

                
                    if col_j + col == 0:
                        axes[row_i+row, col_j+col].set_ylabel(true_label)
                        
                    
                    if col ==num_imgs_axis-1:
                        col =0
                        row+=1
                    else:
                        col +=1
                   
            if empty_img_ids<0:
                raise ValueError("There are more image ids in the dict than number of samples to be plotted")
            
            col_j = col_j + num_imgs_axis
        
        row_i += num_imgs_axis
    
    plt.savefig(path_to_results + "_MS")
    plt.close()

    
def create_dict(classes):
    """
    Setups a dictionary for storing image ids in confusion matrix
    
    """

    output_dict = dict()
    for i in classes:
        output_dict[i] = {}
        for j in classes:
            output_dict[i][j]= []

    return output_dict      

## Confusion Matrix

In [133]:
def plot_CM(y_true, y_pred, path_to_results, path_to_images):
    """
    Plots and save Confusion Matrix
    
    y_true: numpy array of shape (num_of_images,2)
    y_pred: numpy array of shape (num_of_images,2)
    path_to_results: path where plot will be saved

    """

    ConfusionMatrixDisplay.from_predictions(y_pred=y_pred[:,1], y_true =y_true[:,1])

    plt.savefig(path_to_results + "_CM")
    plt.close()

## Create Plots 

In [134]:
def create_plots(y_true, y_pred, path_to_results, path_to_images, plots, name_of_file, training_metric_scores=None, y_pred_probs = None):
    """
    Generates differenet plots
    Parameters:
        y_true: numpy array of shape (folds, num_images, 2)
        y_pred: numoy array of shape (folds, num_images, 2)
        path_to_results: path to the folder where the results will be stored
        plots: dictionary with following structure:
                plots["plots_1"]["name"] = name of plot
                plots["plots_1"]["parameter_1"] = value
                plots["plots_1"]["parameter_2"] = value

                plots["plots_2"]["name"] = name of plot
                plots["plots_2"]["parameter_1"] = value
                plots["plots_2"]["parameter_2"] = value
        
        name_of_file:
        training_metric_scores: (default=None) list of numpy array with each having shape of (folds, metrics, 2, epochs).Each item in list should represent results of a network.

    Returns:
        output_config:

    Additional Notes:
    currently supports CM, LC and MS


    """

    # check whether correct shapes of y_* are provided or not

    if len(y_true.shape) !=3:
        raise ValueError("Shape of y_true is not correct.")

    if len(y_pred.shape) == 3:
        num_folds = y_pred.shape[0]
    else:
        raise ValueError("Shape of y_pred is not correct.")

    # determine whether to plot learning curves or not
    if training_metric_scores is not None:
        lc = True
    else:
        lc = False
   


    for fold_no in range(num_folds):

        # create directory for fold
        path_to_fold = os.path.join(path_to_results, str(fold_no))
        if not os.path.exists(path_to_fold):
            os.mkdir(path_to_fold)

        # create directory for plots
        path_to_plots = os.path.join(path_to_fold, "plots")
        if not os.path.exists(path_to_plots):
            os.mkdir(path_to_plots)

        for pl in plots:

            # if dict has key "learning_curves" but no metric score is provided,
            if pl == "LC" and not lc:
                print("Warning: key'learning_curves' provided in dict 'plots' but metric score not found. Skipping ploting learning curves" )
                continue 


            print(f"Creating Plot: {pl} Fold No: {fold_no}")
            
            plot_name = pl
            if plot_name == "CM":
                fnt_pointer = plot_CM
            elif plot_name == "MS":
                fnt_pointer = plot_MS
            elif plot_name == "LC":
                fnt_pointer = plot_LC
            else:
                raise ValueError("Unknown plot found")
                
            path_to_figs = path_to_plots+f"/{name_of_file}"

            if plot_name == "LC":
                fnt_pointer(metric_score= training_metric_scores[fold_no], path_to_results=path_to_figs, path_to_images=path_to_images)
            else:
                fnt_pointer(y_true=y_true[fold_no], y_pred=y_pred[fold_no], path_to_results=path_to_figs, path_to_images=path_to_images)


    return plots

# Misc

## Load from json 

In [135]:
def load_from_json(path_to_json):
    """
    Load a json file

    Parameters:
        path_to_json: path to json file
    
    Returns:
        dictionary with contents of the json
    """
    with open(path_to_json, 'r') as f:
        dic = json.load(f)
    
    return dic

## Save to json 

In [136]:
def save_to_json(dic, path_to_results):
    """
    Save a dicitonary to a location

    Parameters:
        dic: dictionary to be saved
        path_to_results: path where to save the dictionary

    """
    with open(path_to_results+".json", "w") as fp:
        json.dump(dic, fp, indent=4)

## Generate txt file 

In [137]:
def generate_txt_file(y, path_to_results, name_of_file, y_probs=None):
    """
    Generates a text file with structure as follows:
    file_name_1 label
    file_name_2 label


    Parameters:
        y: numpy array of shape(folds, num_images, 2)
        path_to_results: path to the folder where to store results
        name_of_file (without .txt)
        y_probs: numpy array of shape (folds, num_images, 1)

    """
    

    num_folds = y.shape[0]
    num_images = y.shape[1]


    for fold_no in range(num_folds):

        print(f"Processing Fold No: {fold_no}")

        path_to_fold = os.path.join(path_to_results, str(fold_no))
        if not os.path.exists(path_to_fold):
                os.mkdir(path_to_fold)
        
        # generate file
        path_to_file = path_to_fold + "/" + name_of_file + ".txt"
        open(path_to_file, "w").close()
        with open(path_to_file, "a") as file:
            for img_no in range(num_images):
                file.write(f"{y[fold_no, img_no, 0]} {y[fold_no, img_no, 1]}\n")
        
        # generate files with prob
        if y_probs is not None:
            path_to_prob_file = path_to_fold + "/" + name_of_file + "_with_probs.txt"
            open(path_to_prob_file, "w").close()
            with open(path_to_prob_file, "a") as file:
                for img_no in range(num_images):
                    file.write(f"{y[fold_no, img_no, 0]} {y[fold_no, img_no, 1]} {np.round(y_probs[fold_no, img_no], 4)*100}\n")


## Save results 

In [138]:
def save_results(results, metrics, path_to_results, name_of_file):
    """
    Save results to csv file

    Parameters:
        results: numpy array of shape (num_folds, metrics)
        metrics: numpy array of name of the metrics
        path_to_results: path to the folder where the results will be stored
        name_of_file: file name
       
    """
    df = pd.DataFrame([], columns=["Fold No", "Metric", "Score"])
    num_folds = results.shape[0]
    num_metrics = results.shape[1]

    for fold_no in range(num_folds):
        for metric_no in range(num_metrics):
            temp = pd.DataFrame([[fold_no, metrics[metric_no], results[fold_no, metric_no]]], columns=["Fold No", "Metric", "Score"])
            df = pd.concat([df, temp])

    df.reset_index(inplace=True, drop=True ) 
       
    df.to_csv(path_to_results + "/" + name_of_file +".csv")

## Count Model parameters 

In [139]:
def count_parameters(model):
    """
    
    Counts number of parameters in the model
    
    Parameter:
        model
    
    Returns:
        a dict with keys:
            parameters: 
                name: dict with keys: trainable and total 
            total_trainable_parameters:
            total_parameters
    
    """
    output_config = {}
    output_config["parameters"] = {}
    total_params = 0
    train_params = 0
    
    
    for name, parameter in model.named_parameters():
        params = parameter.numel()
        
        # trainable parameters
        if parameter.requires_grad:
            tr_p = params
            train_params += params 
        else:
            tr_p = 0
        
        # total parameters
        total_params+=params
        
        output_config["parameters"][name] = {}
        output_config["parameters"][name]["trainable"] = params
        output_config["parameters"][name]["total"] = tr_p
       
    output_config["total_trainable_parameters"] =  train_params
    output_config["total_parameters"] =  total_params
    
    
    return output_config

# Prediction

In [140]:
def get_predictions(path_to_results, path_to_images, y, data_transforms, batch_size, device, fold=-2, color="gray"):
    """
    generates predictions
    
    path_to_results: where models are saved
    path_to_images: images dir
    y: (num_images, 1)
    data_transforms: transformation to apply
    fold: prediction for specific fold
    
    
    Returns:
        y_preds_folds: (num_folds, num_images)
        y_preds_probs_folds: (num_folds, num_images, num_classes) 
        y_preds_en: (num_images,1)
        y_preds_probs_ensemble: (num_images, num_classes)
        
    thresholding could be added for binary (>th)/multi(>th for each class) classsifcation
    """
    y_preds_probs_folds=[]
    y_preds_folds = []
    
    sigmoid = torch.nn.Sigmoid()
    softmax = torch.nn.Softmax(dim=1)       
    
    flag = False
    single_neuron = False
    
    for fold_no in sorted(os.listdir(path_to_results)):
        
        path_to_model = os.path.join(path_to_results, fold_no)
        
        if os.path.isdir(path_to_model):
            
            if fold == -2:
                flag = True
            else:
                if fold_no == str(fold):
                    flag = True
                else:
                    flag =False
            
            if flag:
                print(f"Generating predicitons for fold no: {fold_no}")

                # get data
                data = ImageDataset(y, path_to_images, transform=data_transforms, color=color)
                dataloader = DataLoader(data, batch_size=batch_size, shuffle=False)

                # load model
                model_path = os.path.join(path_to_model, "model.pt")
                print (f"Loading model from {model_path}")
                model = torch.load(model_path)
                model.to(device)
                model.eval()

                with torch.no_grad():

                    y_pred_prob=[]
                    y_pred_labels =[]

                    for images, _ in dataloader:
                        images = images.to(device)

                        y_pred = model(images)

                        if y_pred.shape[-1] == 1:
                            # get probs
                            prob = sigmoid(y_pred)

                            # get hard labels
                            labels = np.round(prob.detach().cpu().clone().numpy())

                            single_neuron = True
                        else:
                            #get probs
                            prob = softmax(y_pred)

                            # get hard labels
                            labels = np.argmax(prob.detach().cpu().clone().numpy(), axis=1) 

                        #combine iter results
                        y_pred_prob.extend(prob.detach().cpu().clone().numpy())
                        y_pred_labels.extend(labels)


                    # combine fold results
                    y_preds_folds.append(y_pred_labels)
                    y_preds_probs_folds.append(y_pred_prob)

        
    # get ensemble results
    y_preds_probs_ensemble = np.mean(y_preds_probs_folds, axis=0)
    

    if single_neuron:
        y_preds_en = np.round(y_preds_probs_ensemble).reshape(-1,1)
    else:
        y_preds_en = np.argmax(y_preds_probs_ensemble, axis=1).reshape(-1,1)
        
    return np.array(y_preds_folds), np.array(y_preds_probs_folds), np.array(y_preds_en), np.array(y_preds_probs_ensemble)


In [141]:
def CNN_prediction(path_to_images, path_to_json, save_path):
    """
    Generates prediction for the given data set

    Parameters:
        path_to_images: path to the folder containing images on which prediction will be generated
        path_to_results: path to the folder where models are saved
        save_path: path to the folder where the results will be stored
        data_transform: transformation to apply
        batch_size: batch size

    """

    if not os.path.isdir(path_to_images):
        raise ValueError("'path_to_images' must be a directory")

    if not os.path.exists(save_path):
        os.mkdir(save_path)
        print(f"Created directory {save_path}")
    else:
        print(f"Warning: {save_path} already exists. Content may get overwritten")
    
    # loading training pipeline
    pipeline = load_from_json(path_to_json)

    pipeline["path_to_images"] = path_to_images
    path_to_models = pipeline["path_to_results"]
    
    pipeline["path_to_models"] = path_to_models
    pipeline["path_to_results"] = save_path
    

    del pipeline['path_to_labels']
    
    device = pipeline["device"]
    batch_size= pipeline["batch_size"]
    classes = pipeline["classes"]
    read_img_color = pipeline["read_img_color"]
    
    
    # get transforms
    data_transform = pipeline["data_preprocessing"]["valid"]
    data_transforms = data_preprocessing(data_transform)
    
    # generate y
    y=np.array(sorted(os.listdir(path_to_images))).reshape(-1,1)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using Device: {device}")
  
    print("\nGenerating labels for the data . . . ")
    y_pred, y_pred_probs, y_pred_en, y_pred_probs_en = get_predictions(path_to_results=path_to_models, path_to_images=path_to_images, y=y, data_transforms=data_transforms, batch_size=batch_size, device=device, color=read_img_color)
    
   
    # transform labels and add image ids
    y_pred_tr = np.empty((y_pred.shape[0], y.shape[0], 2), dtype=y.dtype) 
    for nf in range(y_pred.shape[0]):
        y_pred_tr[nf, :, 0] = y[:,0]
        y_pred_tr[nf, :, 1] = label_encoder(y=y_pred[nf,:], classes=classes, to_numbers=False)
    
    
    y_pred_probs_en = np.expand_dims(y_pred_probs_en, axis=0)
    y_pred_en = np.expand_dims (np.concatenate((y, y_pred_en), axis=1), axis=0)
    y_pred_en[0, :, 1] = label_encoder(y=y_pred_en[0, :, 1], classes=classes, to_numbers=False)
    
    print(f"Generated labels for the data successfully. Shape: y_pred: {y_pred_tr.shape} y_pred_probs: {y_pred_probs.shape} y_pred_en: {y_pred_en.shape} y_pred_probs_en: {y_pred_probs_en.shape}")
    
    

    # save pipeline
    print("\nsaving pipeline dictionary to json")
    save_to_json(
        pipeline, 
        save_path +"/pipeline"
        )
    
    # save labels
    print("\nGenerating text file for the data")
    generate_txt_file(
        y=y_pred_tr, 
        path_to_results=save_path, 
        name_of_file="labels",
        y_probs = np.max(y_pred_probs, axis=-1)
        )
    
     # save labels with probs
    print("\nGenerating text file for the data ensemble")
    generate_txt_file(
        y=y_pred_en, 
        path_to_results=save_path, 
        name_of_file="labels_en",
        y_probs = np.max(y_pred_probs_en, axis=-1)
        )
    
    
    pipeline["path_to_models"] = path_to_results
    pipeline["path_to_results"] = save_path
    
    print("\nGenerated Predictions Successfully")

# Training

* for each epoch, train the model and evaluate the model on training and validaiton data using metrics: loss, balanced accuracy, mcc etc

* store evaluation results for each epoch that will be used for learning curves
* save the best model using val loss or some other metric

* generate prediction and prediction probabilities using best model on training and validation data

save all the necessary info

In [142]:
def train_epoch(model, device, dataloader, loss_fnt, optimizer):
    """
    trains a model 
    
    Parameters:
        model: model to be trained
        device: device on which model to be trained
        dataloader: of training dataset
        lost_fnt: loss function
        optimizer:
        
    Returns:
        train_loss, train_balanced_accu, train_mcc 
        unnormalized values (No multiplication of 1/num_images)
    
    """
    
    train_loss = 0
    train_balanced_accu = 0
    train_mcc = 0
    
    sigmoid = torch.nn.Sigmoid()
    
    model.train()

    for images, labels in dataloader:

        images,labels = images.to(device),labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(images)
        
        loss = loss_fnt(output,labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        
        if output.shape[-1] == 1:
            y_pred = torch.round(sigmoid(output.detach().clone())).cpu().numpy()
        else:
            y_pred = np.argmax(output.detach().clone().cpu().numpy(), axis=1)
                    
        train_balanced_accu += accuracy(y_true=labels.detach().clone().cpu().numpy(), y_pred=y_pred, parameters={"type": "balanced"})[0] * images.size(0)
        train_mcc += mcc(y_true=labels.detach().clone().cpu().numpy(), y_pred=y_pred, parameters={})[0] * images.size(0) 

        
    return train_loss, train_balanced_accu, train_mcc

In [143]:
def valid_epoch(model, device, dataloader, loss_fnt):
    """
    Generates results on validation data
    
    Parameters:
        model: model to be used for evaluatioon
        device: device on which model will be evaluated
        dataloader: of validation dataset
        lost_fnt: loss function
        
    Returns:
        valid_loss, valid_balanced_accu, valid_mcc 
        unnormalized values (No multiplication of 1/num_images)
    
    """
    
    valid_loss = 0
    valid_balanced_accu = 0
    valid_mcc = 0
    
    sigmoid = torch.nn.Sigmoid()
    
    model.eval()
    
    with torch.no_grad():
        
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)

            output = model(images)

            loss=loss_fnt(output,labels)

            valid_loss+=loss.item()*images.size(0)
            
            if output.shape[-1] == 1:
                y_pred = torch.round(sigmoid(output.detach().clone())).cpu().numpy()
            else:
                y_pred = np.argmax(output.detach().clone().cpu().numpy(), axis=1)
            
            valid_balanced_accu += accuracy(y_true=labels.cpu().numpy(), y_pred=y_pred, parameters={"type": "balanced"})[0] * images.size(0)
            valid_mcc += mcc(y_true=labels.cpu().numpy(), y_pred=y_pred, parameters={})[0] * images.size(0) 

    return valid_loss, valid_balanced_accu, valid_mcc


In [144]:
def CNN_training(pipeline):
    
    output_config = {}
    
    # set up whether to use cpu or gpu
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using Device: {device}")
    output_config["device"] = device
    
    #get results folder
    path_to_results = pipeline["path_to_results"]
    output_config["path_to_results"] = path_to_results
    if not os.path.exists(path_to_results):
            os.mkdir(path_to_results)
    
    #get iamge folder
    path_to_images = pipeline["path_to_images"]
    output_config["path_to_images"] = path_to_images
    
    # get batch size
    batch_size= pipeline["batch_size"]
    output_config["batch_size"] =batch_size
    
    #reading image in gray scale or rgb
    read_img_color = pipeline["read_img_color"]
    output_config["read_img_color"] = read_img_color
    
    # get batch size
    num_epochs= pipeline["num_epochs"]
    output_config["num_epochs"] =num_epochs
    
    
    # read labels from txt
    path_to_labels = pipeline["path_to_labels"]
    output_config["path_to_labels"] = path_to_labels

    y = np.loadtxt(path_to_labels, dtype=str, delimiter=" ")
    
    # get unique labels and their ratios
    original_labels = {}
    unique_labels, counts = np.unique(y[:,1], return_counts=True)
    print(f"Unique labels found and their frequencies: ")
    for ul, c in zip(unique_labels, counts):
        original_labels[ul] = format(c*100/y.shape[0], '.4f')
        print(f"Label: {ul}, %age: {format(c*100/y.shape[0], '.4f')}")
    output_config["original_labels"] = original_labels
    
    # split data
    split_type = pipeline["split_type"]
    output_config["split_type"] = split_type
    
    if "simple" in split_type:
        test_size = pipeline["test_size"]
        output_config["test_size"] = test_size
        num_folds=1
        y_train, y_valid = split_data(y=y, split_type=split_type, test_size=test_size)
    
    elif "fold" in split_type:
        num_folds = pipeline["num_folds"]
        output_config["num_folds"] = num_folds
        y_train, y_valid = split_data(y=y, split_type=split_type, n_folds=num_folds)
    
    else:
        raise ValueError("Unknown spliting type")
        
    print(f"Split the data successfully. Shape: y_train: {y_train.shape} y_valid: {y_valid.shape}")
        
        
    # number of images in train and valid data
    num_images_train = y_train.shape[1]
    num_images_valid = y_valid.shape[1]
    
    # transform labels
    classes = pipeline["classes"]
    for nf in range(num_folds):
        y_train[nf, :, 1] = label_encoder(y=y_train[nf,:,1], classes=classes, to_numbers=True)
        y_valid[nf, :, 1] = label_encoder(y=y_valid[nf,:,1], classes=classes, to_numbers=True)
    
    output_config["classes"] = classes.tolist()
    output_config["data_preprocessing"]={}
    output_config["data_preprocessing"]["train"] = OrderedDict()
    output_config["data_preprocessing"]["valid"] = OrderedDict()
    
    # get preprocessing transformations for training data
    train_data_transform = pipeline["data_preprocessing"]["train"]
    output_config["data_preprocessing"]["train"] = train_data_transform
    
    train_data_transforms = data_preprocessing(train_data_transform)
    
        
    # get preprocessing transformations for validation data
    valid_data_transform = pipeline["data_preprocessing"]["valid"]
    output_config["data_preprocessing"]["valid"] = valid_data_transform
    
    valid_data_transforms = data_preprocessing(valid_data_transform)
    
     # get input shape for the model
    input_shape = pipeline["input_shape"]
    output_config["input_shape"] = input_shape
    
    # get layers of the model
    layers = pipeline["model"]
    output_config["model"] = layers
    
    use_pretrained_model = pipeline["use_pretrained_model"]
    output_config["use_pretrained_model"] = use_pretrained_model
    
    #get loss settings
    loss_param = pipeline["loss"]
    output_config["loss"] = loss_param
    
    # get optimizer settings
    optim_parameters = pipeline["optimizer"]
    output_config["optimizer"] =optim_parameters 
    
    #get metrics
    metrics = pipeline["metrics"]
    output_config["metrics"] =metrics
    
    # get plots
    plots = pipeline["plots"]
    output_config["plots"] =plots
    
    
    # for saving scores of each fold
    train_loss_fold, train_accu_fold,train_mcc_fold = [],[],[]
    valid_loss_fold, valid_accu_fold,valid_mcc_fold = [],[],[]
    
    # for saving scores of best model of each model
    train_loss_fold_best, train_accu_fold_best,train_mcc_fold_best = [],[],[]
    valid_loss_fold_best, valid_accu_fold_best,valid_mcc_fold_best = [],[],[]
    best_model_epoch_no_fold = []
    
    train_labels ={}
    valid_labels={}
    class_weights_dict = {}
    
    for fold_no in range(num_folds):
        
        path_to_fold = os.path.join(path_to_results, str(fold_no))
        if not os.path.exists(path_to_fold):
            os.mkdir(path_to_fold)
        
        # get training data
        train_data = ImageDataset(y_train[fold_no], path_to_images, transform=train_data_transforms,color=read_img_color)
        train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        
        # get validation data
        valid_data = ImageDataset(y_valid[fold_no], path_to_images, transform=valid_data_transforms, color=read_img_color)
        valid_dataloader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
        
        # load model
        model = Model(input_shape,layers, use_pretrained_model)
        model.to(device)
        model_parameters = count_parameters(model)
        
        if fold_no == 0:
            output_config["model_parameters"] = model_parameters
        
        trainable_params = model_parameters["total_trainable_parameters"]
        total_params = model_parameters["total_parameters"]
    

        print(f"\nFold No: {fold_no+1} Model: \n {model}")
        print(f"Total trainable parameters: {trainable_params} Total parameters: {total_params}")
        
        # get ratio of each class in y_train
        train_labels[str(fold_no)] = {}
        unique_labels, counts = np.unique(y_train[fold_no,:,1], return_counts=True)
        print(f"\nFold No: {fold_no+1} Training data: Unique labels: ")
        for ul, c in zip(unique_labels, counts):
                train_labels[str(fold_no)][ul] = format(c*100/num_images_train, '.4f')
                print(f"Label: {ul}, %age: {format(c*100/num_images_train, '.4f')}")

        # get ratio of each class in y_valid
        valid_labels[str(fold_no)] = {}
        unique_labels, counts =np.unique(y_valid[fold_no, :,1], return_counts=True)
        print(f"\nFold No: {fold_no+1} Validation data: Unique labels: ")
        for ul, c in zip(unique_labels, counts):
            valid_labels[str(fold_no)][ul] = format(c*100/num_images_valid, '.4f')
            print(f"Label: {ul}, %age: {format(c*100/num_images_valid, '.4f')}")
        
        # setup loss
        if loss_param["use_single_neuron"]:
            class_weights = [float(train_labels[str(fold_no)]['0']) / float(train_labels[str(fold_no)]['1'])]
            
            loss_fnt = setup_loss({**loss_param, **{"class_weights": class_weights, "device": device}})
            print(f"\nFold No: {fold_no+1} loss: {loss_fnt} weights: {loss_fnt.pos_weight}")
        
        else:
            class_weights = []
            for cl in range(len(classes)):
                class_weights.append(1 - float(train_labels[str(fold_no)][str(cl)])/100 )

            loss_fnt = setup_loss({**loss_param, **{"class_weights": class_weights, "device": device}})
            print(f"\nFold No: {fold_no+1} loss: {loss_fnt} weights: {loss_fnt.weight}")
        
        class_weights[fold_no] = class_weights

        # setup optimizer
        optimizer = setup_optimizer(model.parameters(), optim_parameters)
        print(f"\nFold No: {fold_no+1} Optimizer: \n {optimizer}")
          
        # for saving scores of each epoch of best model
        train_loss_epoch, train_accu_epoch,train_mcc_epoch = [],[],[]
        valid_loss_epoch, valid_accu_epoch,valid_mcc_epoch = [],[],[]
        
        for epoch in range(num_epochs):
            
            print(f"Training :Fold No: {fold_no+1}/{num_folds} Epoch:{epoch+1}/{num_epochs} . . .")
            
            train_loss, train_accu, train_mcc = train_epoch(model, device, train_dataloader, loss_fnt, optimizer)
            valid_loss, valid_accu, valid_mcc = valid_epoch(model, device, valid_dataloader, loss_fnt)
            
            train_loss, train_accu, train_mcc = train_loss/num_images_train, train_accu/num_images_train, train_mcc/num_images_train
            valid_loss, valid_accu, valid_mcc = valid_loss/num_images_valid, valid_accu/num_images_valid, valid_mcc/num_images_valid
            
            print(f'Evaluated:Fold No: {fold_no+1}/{num_folds} Epoch:{epoch+1}/{num_epochs} Training: Loss: {np.around(train_loss, 4)}, Bal_accu: {np.around(train_accu, 4)}, mcc: {np.around(train_mcc, 4)} Validation: Loss: {np.around(valid_loss, 4)}, Bal_accu: {np.around(valid_accu, 4)}, mcc: {np.around(valid_mcc, 4)}')
             
            # save best model
            if epoch == 0:
                print(f"Saving Model of first epoch")
                
                torch.save(model, os.path.join(path_to_fold, "model.pt"))
                
                # save best model scores
                best_model_epoch_no = epoch
                train_loss_epoch_best = train_loss
                train_accu_epoch_best = train_accu
                train_mcc_epoch_best = train_mcc

                valid_loss_epoch_best = valid_loss
                valid_accu_epoch_best = valid_accu
                valid_mcc_epoch_best = valid_mcc   

            elif  valid_loss_epoch_best > valid_loss:
                
                print(f"Validation Loss score improved. Saving Model.")
                
                torch.save(model, os.path.join(path_to_fold, "model.pt"))
                
                # save best model scores
                best_model_epoch_no = epoch
                train_loss_epoch_best = train_loss
                train_accu_epoch_best = train_accu
                train_mcc_epoch_best = train_mcc

                valid_loss_epoch_best = valid_loss
                valid_accu_epoch_best = valid_accu
                valid_mcc_epoch_best = valid_mcc   

            else:
                print("Validaiton Loss score did not improve.")
      
            # save scores for learning curves
            train_loss_epoch.append(train_loss)
            train_accu_epoch.append(train_accu)
            train_mcc_epoch.append(train_mcc)
            
            valid_loss_epoch.append(valid_loss)
            valid_accu_epoch.append(valid_accu)
            valid_mcc_epoch.append(valid_mcc)
            
        print(f"\nFold No: {fold_no+1}/{num_folds} Best Validation Loss and balanced accuracy recorded on epoch {best_model_epoch_no+1}: {valid_loss_epoch_best} {valid_accu_epoch_best}")

        # save best model for each fold
        best_model_epoch_no_fold.append(best_model_epoch_no)
        
        train_loss_fold_best.append(train_loss_epoch_best)
        train_accu_fold_best.append(train_accu_epoch_best)
        train_mcc_fold_best.append(train_mcc_epoch_best)

        valid_loss_fold_best.append(valid_loss_epoch_best)
        valid_accu_fold_best.append(valid_accu_epoch_best)
        valid_mcc_fold_best.append(valid_mcc_epoch_best)
        
        # save scores for each fold
        train_loss_fold.append(train_loss_epoch)
        train_accu_fold.append(train_accu_epoch)
        train_mcc_fold.append(train_mcc_epoch)

        valid_loss_fold.append(valid_loss_epoch)
        valid_accu_fold.append(valid_accu_epoch)
        valid_mcc_fold.append(valid_mcc_epoch)
    
    
    #convert to numpy
    train_loss_fold = np.array(train_loss_fold)
    train_accu_fold = np.array(train_accu_fold)
    train_mcc_fold = np.array(train_mcc_fold)
    
    valid_loss_fold = np.array(valid_loss_fold)
    valid_accu_fold = np.array(valid_accu_fold)
    valid_mcc_fold = np.array(valid_mcc_fold)
    
    
    
    # generate learning curves array
    score_for_learning_curves = np.empty((num_folds, 3,2, num_epochs), dtype=train_loss_fold.dtype)
    for nf in range(num_folds):
        score_for_learning_curves[nf, 0, 0, :] = train_loss_fold[nf]
        score_for_learning_curves[nf, 1, 0, :] = train_accu_fold[nf]
        score_for_learning_curves[nf, 2, 0, :] = train_mcc_fold[nf]
        
        score_for_learning_curves[nf, 0, 1, :] = valid_loss_fold[nf]
        score_for_learning_curves[nf, 1, 1, :] = valid_accu_fold[nf]
        score_for_learning_curves[nf, 2, 1, :] = valid_mcc_fold[nf]
    
    print(f"\nTrained the model for all folds. Shape: train: loss: {train_loss_fold.shape} accu: {train_accu_fold.shape} mcc: {train_mcc_fold.shape} valid: loss: {valid_loss_fold.shape} accu: {valid_accu_fold.shape} mcc: {valid_mcc_fold.shape} Learning scores: {score_for_learning_curves.shape}")
    
    output_config["train_labels"] = train_labels
    output_config["valid_labels"] = valid_labels
    
    output_config["results"] = {}
    output_config["results"]["best_model"]={}
    output_config["results"]["best_model"]["epoch_number"]=best_model_epoch_no_fold
    output_config["results"]["best_model"]["train"]={}
    output_config["results"]["best_model"]["train"]["loss"]=train_loss_fold_best
    output_config["results"]["best_model"]["train"]["balanced_accuracy"]=train_accu_fold_best
    output_config["results"]["best_model"]["train"]["mcc"]=train_mcc_fold_best
    output_config["results"]["best_model"]["valid"]={}
    output_config["results"]["best_model"]["valid"]["loss"]=valid_loss_fold_best
    output_config["results"]["best_model"]["valid"]["balanced_accuracy"]=valid_accu_fold_best
    output_config["results"]["best_model"]["valid"]["mcc"]=valid_mcc_fold_best
    
    # generate predicitons
    train_pred = []
    train_pred_prob = []
    valid_pred = []
    valid_pred_prob = []
    for fn in range(num_folds):
        print("\nGenerating Predictions on training data")
        train_pred_fn, train_pred_prob_fn, _,_= get_predictions(path_to_results=path_to_results, path_to_images=path_to_images, y=y_train[fn], data_transforms=valid_data_transforms, fold=fn, batch_size=batch_size, device=device, color=read_img_color)
        print("Generating Predictions on validation data")
        valid_pred_fn, valid_pred_prob_fn, _,_= get_predictions(path_to_results=path_to_results, path_to_images=path_to_images, y=y_valid[fn], data_transforms=valid_data_transforms, fold=fn, batch_size=batch_size, device=device, color=read_img_color)
        
        train_pred_fn = label_encoder(y=train_pred_fn, classes=classes, to_numbers=False)
        valid_pred_fn = label_encoder(y=valid_pred_fn, classes=classes, to_numbers=False)
     
        
        train_pred.append([y_train[fn, :, 0], train_pred_fn])
        train_pred_prob.append(train_pred_prob_fn[0])
        valid_pred.append([y_valid[fn, :, 0],valid_pred_fn])
        valid_pred_prob.append(valid_pred_prob_fn[0])
        

    train_pred =np.array(train_pred).swapaxes(-1,-2)
    train_pred_prob =np.array(train_pred_prob)
    valid_pred =np.array(valid_pred).swapaxes(-1,-2)
    valid_pred_prob =np.array(valid_pred_prob)
    
    print(f"\nGenerated predictions on training and validation dataset. Shape: train: labels: {train_pred.shape} prob: {train_pred_prob.shape} valid: labels: {valid_pred.shape} prob: {valid_pred_prob.shape}")    
    
    # transform int labels to name of classes
    for nf in range(num_folds):
        y_train[nf, :, 1] = label_encoder(y=y_train[nf,:,1], classes=classes, to_numbers=False)
        y_valid[nf, :, 1] = label_encoder(y=y_valid[nf,:,1], classes=classes, to_numbers=False)
        
    
     # generate predicitons on complete dataset
    print("\nGenerating predicitons on complete dataset. . . ")
    
    # transform name of classes to int
    y[:, 1] = label_encoder(y=y[:,1], classes=classes, to_numbers=True)
    
    y_pred_comp, y_pred_prob_comp, y_pred_en, y_pred_prob_en = get_predictions(path_to_results=path_to_results, path_to_images=path_to_images, y=y, data_transforms=valid_data_transforms, device=device, batch_size=batch_size, color=read_img_color)
    
    
    # transform labels and add image ids
    y_pred_com = np.empty((num_folds, y.shape[0], 2), dtype=y.dtype) 
    for nf in range(num_folds):
        y_pred_com[nf, :, 0] = y[:,0]
        y_pred_com[nf, :, 1] = label_encoder(y=y_pred_comp[nf], classes=classes, to_numbers=False)
    
    y_pred_prob_com = y_pred_prob_comp
        
    y_pred_en = np.expand_dims (np.concatenate((y[:,0].reshape(-1,1), y_pred_en), axis=1), axis=0)
    y_pred_en[0, :, 1] = label_encoder(y=y_pred_en[0, :, 1], classes=classes, to_numbers=False)
    
    y_pred_prob_en = np.expand_dims(y_pred_prob_en, axis=0)   
    
    y = np.repeat(y[None, :,:], num_folds, axis=0)
    
    print(f"\nGenerated predicitons on complete dataset. Shape: Folds: labels: {y_pred_com.shape} probs: {y_pred_prob_com.shape} Ensembles: labels: {y_pred_en.shape} probs: {y_pred_prob_en.shape}")
    
    
    #evaluate metrics on training data
    print("\nEvaluating Metrics on training data . . . ")
    metrics_train, metrics_train_config, metrics_train_list = evaluate_metrics(
        y_true=y_train, 
        y_pred=train_pred, 
        metrics=metrics,
        y_pred_probs = train_pred_prob
        )
    print("\nResults")


    for fold_no in range(num_folds):
        for met_no, met in enumerate(metrics_train_list):
            print(f"Fold No: {fold_no} Metric: {met}  Score: {np.around(metrics_train[fold_no, met_no], 4)} ")

    print(f"Evaluated Metrics on the training data successfully. Shape:{metrics_train.shape}")

    #evaluate metrics on validataion data
    print("\nEvaluating Metrics on validation data . . . ")
    metrics_valid, _, metrics_valid_list = evaluate_metrics(
        y_true=y_valid, 
        y_pred=valid_pred, 
        metrics=metrics,
        y_pred_probs = valid_pred_prob
        )
    print("\nResults")
    
    for fold_no in range(num_folds):
        for met_no, met in enumerate(metrics_valid_list):
            print(f"Fold No: {fold_no} Metric: {met}  Score: {np.around(metrics_valid[fold_no, met_no],4)} ")

    print(f"Evaluated Metrics on the validation data successfully. Shape: {metrics_valid.shape}")
    
    print("\nEvaluating Metrics on complete data . . . ")
    # transform int labels to name of classes
    for nf in range(num_folds):
        y[nf,:, 1] = label_encoder(y=y[nf,:,1], classes=classes, to_numbers=False)
    
    metrics_com, metrics_com_config, metrics_com_list = evaluate_metrics(
        y_true=y, 
        y_pred=y_pred_com, 
        metrics=metrics,
        y_pred_probs = y_pred_prob_com
        )
    print("\nResults")
    
    for nf in range(num_folds):
        for met_no, met in enumerate(metrics_com_list):
            print(f"Fold No: {nf} Metric: {met}  Score: {np.around(metrics_com[nf, met_no],4)} ")
            
    print(f"Evaluated Metrics on the complete dataset successfully. Shape:{metrics_com.shape}")
    
    print("\nEvaluating Metrics on complete dataset using ensembles . . . ") 
    metrics_en, metrics_en_config, metrics_en_list = evaluate_metrics(
        y_true=np.expand_dims(y[0], axis=0), 
        y_pred=y_pred_en, 
        metrics=metrics,
        y_pred_probs = y_pred_prob_en
        )
    print("\nResults")
    
    for met_no, met in enumerate(metrics_en_list):
            print(f"Fold No: {0} Metric: {met}  Score: {np.around( metrics_en[0, met_no], 4)} ")
            
    print(f"Evaluated Metrics on the complete dataset using ensemble successfully. Shape:{metrics_en.shape}")

            
    # create plots
    print("\nCreating Plots for training data . . .")
    plots_train_config = create_plots(
        y_true=y_train, 
        y_pred=train_pred, 
        plots= plots, 
        path_to_results=path_to_results,
        path_to_images=path_to_images,
        name_of_file = "train",
        training_metric_scores=score_for_learning_curves,
        y_pred_probs = train_pred_prob
        )
    print("Created Plots for training data")

    print("\nCreating Plots for validation data")
    _ = create_plots(
        y_true=y_valid, 
        y_pred=valid_pred, 
        plots= plots, 
        path_to_results=path_to_results,
        path_to_images=path_to_images, 
        name_of_file = "valid",
        y_pred_probs = valid_pred_prob
        )
    print("Created Plots for validation data")
    
    print("\nCreating Plots for complete dataset")
    _ = create_plots(
        y_true=y, 
        y_pred=y_pred_com, 
        plots= plots, 
        path_to_results=path_to_results,
        path_to_images=path_to_images, 
        name_of_file = "complete",
        y_pred_probs = y_pred_prob_com
        )
    print("Created Plots for complete dataset")
    
    print("\nCreating Plots for ensemble dataset")
    _ = create_plots(
        y_true=np.expand_dims(y[0], axis=0), 
        y_pred=y_pred_en, 
        plots= plots, 
        path_to_results=path_to_results,
        path_to_images=path_to_images, 
        name_of_file = "ensemble",
        y_pred_probs = y_pred_prob_en
        )
    print("Created Plots for ensemble data")

    # save pipeline
    print("\nsaving pipeline dictionary to json")
    save_to_json(
        output_config, 
        os.path.join(path_to_results, "training_pipeline")
        )
    
        # save labels
    print("\nGenerating text file for training data")
    generate_txt_file(
        y=train_pred, 
        path_to_results=path_to_results, 
        name_of_file="train",
        y_probs = np.max(train_pred_prob, axis=-1)
        )

    print("\nGenerating text file for validation data")
    generate_txt_file(
        y=valid_pred, 
        path_to_results=path_to_results,  
        name_of_file="valid",
        y_probs = np.max(valid_pred_prob, axis=-1)
        )
    
    print("\nGenerating text file for complete data")
    generate_txt_file(
        y=y_pred_com, 
        path_to_results=path_to_results,  
        name_of_file="complete",
        y_probs = np.max(y_pred_prob_com, axis=-1)
        )
    
    print("\nGenerating text file for ensemble data")
    generate_txt_file(
        y=y_pred_en, 
        path_to_results=path_to_results,  
        name_of_file="ensemble",
        y_probs = np.max(y_pred_prob_en, axis=-1)
        )
    
    # save metrics train and metrics_valid
    print("\nSaving training results ")
    save_results(
        results=metrics_train,
        metrics=metrics_train_list,
        path_to_results=path_to_results,
        name_of_file="train"
    )

    print("\nSaving validation results ")
    save_results(
        results=metrics_valid,
        metrics=metrics_valid_list,
        path_to_results=path_to_results,
        name_of_file="valid"
    )
    
    print("\nSaving complete data results ")
    save_results(
        results=metrics_com,
        metrics=metrics_com_list,
        path_to_results=path_to_results,
        name_of_file="complete"
    )
    
    print("\nSaving ensemble results ")
    save_results(
        results=metrics_en,
        metrics=metrics_en_list,
        path_to_results=path_to_results,
        name_of_file="ensemble"
    )

    print("\nTraining Completed\n")
    




# Save
* the model and all the info (model, parameters, results etc)

# Evaluate Metrics
Using the best model, evaluate the metircs and save the find results in csv file

# Plots:
* learning curve
* metrics
* confusion matrix
* misidentified samples
* ROC
* maybe weights

save them

# Prediction
load the best model, generate prediction on train, test and noisy test dataset and create two txt files:
- one with file name and label (like the one provided)
- one with file name, label and probability

# Additional notes:
* set up excel sheet for finding optimal hyperparameers in organized manner
* set up excel for tracking results of different models in organized manner
* or just find some function that finds optimal hyperparameters

* Results may not stay saved if kaggle session is closed. Either download them in timely manner or find a way to save them

# Results Directory

In [145]:
run_name = 'ResNet50'

path_to_result ='/home/ahmad/Documents/TUHH/Semester 3/Intelligent Systems in Medicine/Project/Classification-of-different-dieases-using-ML-and-DL/results/Phase2_results/local_comp'
if not os.path.exists(path_to_result):
    os.mkdir(path_to_result)
    
path_to_results = os.path.join(path_to_result, run_name)
if not os.path.exists(path_to_results):
    os.mkdir(path_to_results)
    
path_to_binary_results = os.path.join(path_to_results, "binary")
if not os.path.exists(path_to_binary_results):
    os.mkdir(path_to_binary_results)

path_to_multiclass_results = os.path.join(path_to_results, "multi")
if not os.path.exists(path_to_multiclass_results):
    os.mkdir(path_to_multiclass_results)


# Raw Data Directory

In [146]:
# path to data folders
path_to_data = "/home/ahmad/Documents/TUHH/Semester 3/Intelligent Systems in Medicine/Project/Classification-of-different-dieases-using-ML-and-DL/data/raw_data"

# training data
path_to_train_data = os.path.join(path_to_data, "train")
path_to_binary_labels = os.path.join(path_to_data, "train_binary.txt")
path_to_multi_labels = os.path.join(path_to_data, "train_multi.txt")

# testing data
path_to_test_data = os.path.join(path_to_data, "test")

# noisy test data
path_to_noisy_test_data = os.path.join(path_to_data, "noisy_test")

In [147]:
num_train_data = len(os.listdir(path_to_train_data))
num_test_data = len(os.listdir(path_to_test_data))
num_noisy_test_data = len(os.listdir(path_to_noisy_test_data))
print(f"Number of images in: training data: {num_train_data} test data: {num_test_data} noisy test data: {num_test_data}")

Number of images in: training data: 16930 test data: 4235 noisy test data: 4235


# Binary Classification

## Train 

In [148]:
#--------------------------------------Pipeline-------------------------------

pipeline_binary = {}
pipeline_binary["path_to_results"] = os.path.join(path_to_binary_results, "train")

pipeline_binary["batch_size"] = 32
pipeline_binary["num_epochs"] = 20

# for model
pipeline_binary["input_shape"] = (224, 224,3)#for mobile net 

# path to folder containing images 
pipeline_binary["path_to_images"] = path_to_train_data
# path to labels.txt
pipeline_binary["path_to_labels"] = path_to_binary_labels

# split data
pipeline_binary["split_type"] = "simpleStratified" #"simple", "simpleStratified", "kfold", "kfoldStratified"
pipeline_binary["test_size"] = 0.3

# names of the class
pipeline_binary["classes"] = np.array(["NO_COVID", "COVID"])
pipeline_binary["use_pretrained_model"]= True
pipeline_binary["read_img_color"] = "rgb"

# ---------------------------------set up data preprocessing methods and parameters------------------------------------
pipeline_binary["data_preprocessing"] = {}
pipeline_binary["data_preprocessing"]["train"] = OrderedDict()
pipeline_binary["data_preprocessing"]["train"]["normalize_0_1"] = {}
pipeline_binary["data_preprocessing"]["train"]["normalize_0_1"]["name"] = "normalize"
pipeline_binary["data_preprocessing"]["train"]["normalize_0_1"]["mean"] = 0
pipeline_binary["data_preprocessing"]["train"]["normalize_0_1"]["std"] = 255

pipeline_binary["data_preprocessing"]["train"]["random_rotation"] = {}
pipeline_binary["data_preprocessing"]["train"]["random_rotation"]["name"] =  "random_rotation"
pipeline_binary["data_preprocessing"]["train"]["random_rotation"]["degrees"] = 90
pipeline_binary["data_preprocessing"]["train"]["random_rotation"]["expand"] = True

pipeline_binary["data_preprocessing"]["train"]["random_crop"] = {}
pipeline_binary["data_preprocessing"]["train"]["random_crop"]["name"] = "random_resized_crop"
pipeline_binary["data_preprocessing"]["train"]["random_crop"]["output_shape"] = 224

pipeline_binary["data_preprocessing"]["train"]["horizontal_flip"] = {}
pipeline_binary["data_preprocessing"]["train"]["horizontal_flip"]["name"] = "random_horizontal_flip"

pipeline_binary["data_preprocessing"]["train"]["vertical_flip"] = {}
pipeline_binary["data_preprocessing"]["train"]["vertical_flip"]["name"] = "random_vertical_flip"

pipeline_binary["data_preprocessing"]["train"]["normalize_model_values"] = {}
pipeline_binary["data_preprocessing"]["train"]["normalize_model_values"]["name"] = "normalize"
pipeline_binary["data_preprocessing"]["train"]["normalize_model_values"]["mean"] = [0.485, 0.456, 0.406]
pipeline_binary["data_preprocessing"]["train"]["normalize_model_values"]["std"] = [0.229, 0.224, 0.225]


pipeline_binary["data_preprocessing"]["valid"] = OrderedDict()

pipeline_binary["data_preprocessing"]["valid"]["normalize_0_1"] = {}
pipeline_binary["data_preprocessing"]["valid"]["normalize_0_1"]["name"] = "normalize"
pipeline_binary["data_preprocessing"]["valid"]["normalize_0_1"]["mean"] = 0
pipeline_binary["data_preprocessing"]["valid"]["normalize_0_1"]["std"] = 255

pipeline_binary["data_preprocessing"]["valid"]["center_crop"] = {}
pipeline_binary["data_preprocessing"]["valid"]["center_crop"]["name"] = "center_crop"
pipeline_binary["data_preprocessing"]["valid"]["center_crop"]["output_shape"] = (224,224)

pipeline_binary["data_preprocessing"]["valid"]["normalize_model_values"] = {}
pipeline_binary["data_preprocessing"]["valid"]["normalize_model_values"]["name"] = "normalize"
pipeline_binary["data_preprocessing"]["valid"]["normalize_model_values"]["mean"] = [0.485, 0.456, 0.406]
pipeline_binary["data_preprocessing"]["valid"]["normalize_model_values"]["std"] = [0.229, 0.224, 0.225]



# ---------------------------------set up networks and parameters------------------------------------

pipeline_binary["model"] = OrderedDict()
pipeline_binary["model"]["resnet50"] = {}
pipeline_binary["model"]["resnet50"]["name"] = "resnet50"
pipeline_binary["model"]["resnet50"]["num_output_neurons"] = 2
#pipeline_binary["model"]["mobile_net"]["num_layers_to_train"] = 5


#pipeline_binary["model"]["conv1"] = {}
#pipeline_binary["model"]["conv1"]["name"] = "conv"
#pipeline_binary["model"]["conv1"]["number_of_kernels"] = 32
#pipeline_binary["model"]["conv1"]["kernel_size"] = 7
#pipeline_binary["model"]["lrelu1"] = {}
#pipeline_binary["model"]["lrelu1"]["name"] = "lrelu"
#pipeline_binary["model"]["maxpool1"] = {}
#pipeline_binary["model"]["maxpool1"]["name"] = "max_pool"
#pipeline_binary["model"]["maxpool1"]["kernel_size"] = 2


#pipeline_binary["model"]["conv2"] = {}
#pipeline_binary["model"]["conv2"]["name"] = "conv"
#pipeline_binary["model"]["conv2"]["number_of_kernels"] = 64
#pipeline_binary["model"]["conv2"]["kernel_size"] = 5
#pipeline_binary["model"]["lrelu2"] = {}
#pipeline_binary["model"]["lrelu2"]["name"] = "lrelu"
#pipeline_binary["model"]["maxpool2"] = {}
#pipeline_binary["model"]["maxpool2"]["name"] = "max_pool"
#pipeline_binary["model"]["maxpool2"]["kernel_size"] = 2


#pipeline_binary["model"]["conv3"] = {}
#pipeline_binary["model"]["conv3"]["name"] = "conv"
#pipeline_binary["model"]["conv3"]["number_of_kernels"] = 128
#pipeline_binary["model"]["conv3"]["kernel_size"] = 3
#pipeline_binary["model"]["lrelu3"] = {}
#pipeline_binary["model"]["lrelu3"]["name"] = "lrelu"
#pipeline_binary["model"]["maxpool3"] = {}
#pipeline_binary["model"]["maxpool3"]["name"] = "max_pool"
#pipeline_binary["model"]["maxpool3"]["kernel_size"] = 3

#pipeline_binary["model"]["flatten4"] = {}
#pipeline_binary["model"]["flatten4"]["name"] = "flatten"

#pipeline_binary["model"]["linear5"] = {}
#pipeline_binary["model"]["linear5"]["name"] = "linear"
#pipeline_binary["model"]["linear5"]["neurons"] = 256
#pipeline_binary["model"]["lrelu5"] = {}
#pipeline_binary["model"]["lrelu5"]["name"] = "lrelu"

#pipeline_binary["model"]["linear6"] = {}
#pipeline_binary["model"]["linear6"]["name"] = "linear"
#pipeline_binary["model"]["linear6"]["neurons"] = 128
#pipeline_binary["model"]["lrelu6"] = {}
#pipeline_binary["model"]["lrelu6"]["name"] = "lrelu"

#pipeline_binary["model"]["output"] = {}
#pipeline_binary["model"]["output"]["name"] = "linear"
#pipeline_binary["model"]["output"]["neurons"] = 2

pipeline_binary["optimizer"] = {}
pipeline_binary["optimizer"]["name"] = "adam"

pipeline_binary["loss"] = {}
pipeline_binary["loss"]["type"] = "cross_entropy"
pipeline_binary["loss"]["use_weighted_loss"] = True
pipeline_binary["loss"]["use_single_neuron"] = False

#---------------------------------------------set up evaluation metrics and parameters------------------------


pipeline_binary["metrics"] = {}

# accuracy
pipeline_binary["metrics"]["simple_accuracy"] = {}
pipeline_binary["metrics"]["simple_accuracy"]["name"] = "accuracy"
pipeline_binary["metrics"]["simple_accuracy"]["type"] = "simple"  

pipeline_binary["metrics"]["balanced_accuracy"] = {}
pipeline_binary["metrics"]["balanced_accuracy"]["name"] = "accuracy"
pipeline_binary["metrics"]["balanced_accuracy"]["type"] = "balanced"

# precision
pipeline_binary["metrics"]["precision"] = {}
pipeline_binary["metrics"]["precision"]["name"] = "precision"
pipeline_binary["metrics"]["precision"]["class_result"] = "COVID"

# recall
pipeline_binary["metrics"]["sensitivity"] = {}
pipeline_binary["metrics"]["sensitivity"]["name"] = "sensitivity"
pipeline_binary["metrics"]["sensitivity"]["class_result"]  = "COVID"

# F1 score
pipeline_binary["metrics"]["f1_score"] = {}
pipeline_binary["metrics"]["f1_score"]["name"] = "F1_score" 
pipeline_binary["metrics"]["f1_score"]["class_result"] = "COVID"

# mcc
pipeline_binary["metrics"]["mcc"] = {}
pipeline_binary["metrics"]["mcc"]["name"] ="mcc" 

#------------------------------------------------Create Plots --------------------------
pipeline_binary["plots"] = ["CM", "LC", "MS"]


In [149]:
start = time.time()
CNN_training(pipeline_binary)
end = time.time()
print("Time taken to train the model : ", datetime.timedelta(seconds=end-start))

Using Device: cpu
Unique labels found and their frequencies: 
Label: COVID, %age: 17.0821
Label: NO_COVID, %age: 82.9179
Splitting data using Simple Stratified with 70.0-30.0 ratio
Split the data successfully. Shape: y_train: (1, 11851, 2) y_valid: (1, 5079, 2)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/ahmad/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%



Fold No: 1 Model: 
 Model(
  (seq_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

## test data 

In [150]:
path_to_json = os.path.join(path_to_binary_results, "train", "training_pipeline.json")
save_path = os.path.join(path_to_binary_results, "test")

start = time.time()
CNN_prediction(path_to_test_data, path_to_json, save_path)
end = time.time()
print("Time taken to generate predictions on test data : ", datetime.timedelta(seconds=end-start))

Created directory /home/ahmad/Documents/TUHH/Semester 3/Intelligent Systems in Medicine/Project/Classification-of-different-dieases-using-ML-and-DL/results/Phase2_results/local_comp/ResNet50/binary/test
Using Device: cpu

Generating labels for the data . . . 
Generating predicitons for fold no: 0
Loading model from /home/ahmad/Documents/TUHH/Semester 3/Intelligent Systems in Medicine/Project/Classification-of-different-dieases-using-ML-and-DL/results/Phase2_results/local_comp/ResNet50/binary/train/0/model.pt
Generated labels for the data successfully. Shape: y_pred: (1, 4235, 2) y_pred_probs: (1, 4235, 2) y_pred_en: (1, 4235, 2) y_pred_probs_en: (1, 4235, 2)

saving pipeline dictionary to json

Generating text file for the data
Processing Fold No: 0

Generating text file for the data ensemble
Processing Fold No: 0

Generated Predictions Successfully
Time taken to generate predictions on test data :  0:02:47.010450


## noisy test data 

In [151]:
path_to_json = os.path.join(path_to_binary_results, "train", "training_pipeline.json")
save_path = os.path.join(path_to_binary_results, "noisy_test")

start = time.time()
CNN_prediction(path_to_noisy_test_data, path_to_json, save_path)
end = time.time()
print("Time taken to generate predictions on noisy test data : ", datetime.timedelta(seconds=end-start))

Created directory /home/ahmad/Documents/TUHH/Semester 3/Intelligent Systems in Medicine/Project/Classification-of-different-dieases-using-ML-and-DL/results/Phase2_results/local_comp/ResNet50/binary/noisy_test
Using Device: cpu

Generating labels for the data . . . 
Generating predicitons for fold no: 0
Loading model from /home/ahmad/Documents/TUHH/Semester 3/Intelligent Systems in Medicine/Project/Classification-of-different-dieases-using-ML-and-DL/results/Phase2_results/local_comp/ResNet50/binary/train/0/model.pt
Generated labels for the data successfully. Shape: y_pred: (1, 4235, 2) y_pred_probs: (1, 4235, 2) y_pred_en: (1, 4235, 2) y_pred_probs_en: (1, 4235, 2)

saving pipeline dictionary to json

Generating text file for the data
Processing Fold No: 0

Generating text file for the data ensemble
Processing Fold No: 0

Generated Predictions Successfully
Time taken to generate predictions on noisy test data :  0:02:47.988636


# Multiclass Classification

## Train 

In [152]:
pipeline_multi = {}
pipeline_multi["path_to_results"] = os.path.join(path_to_multiclass_results, "train")

pipeline_multi["batch_size"] = 32
pipeline_multi["num_epochs"] = 20

# for model
pipeline_multi["input_shape"] = (224, 224) 

# path to folder containing images 
pipeline_multi["path_to_images"] = path_to_train_data
# path to labels.txt
pipeline_multi["path_to_labels"] = path_to_multi_labels

# split data
pipeline_multi["split_type"] ="simpleStratified" #"kfoldStratified" #"simpleStratified" #"simple", "simpleStratified", "kfold", "kfoldStratified"
#pipeline_multi["num_folds"] = 2
pipeline_multi["test_size"] = 0.3

# names of the class
pipeline_multi["classes"] = np.array(["Normal", "COVID", "pneumonia", "Lung_Opacity"])
pipeline_multi["use_pretrained_model"]= True
pipeline_multi["read_img_color"] = "rgb"

# ---------------------------------set up data preprocessing methods and parameters------------------------------------
pipeline_multi["data_preprocessing"] = {}
pipeline_multi["data_preprocessing"]["train"] = OrderedDict()
pipeline_multi["data_preprocessing"]["train"]["normalize_0_1"] = {}
pipeline_multi["data_preprocessing"]["train"]["normalize_0_1"]["name"] = "normalize"
pipeline_multi["data_preprocessing"]["train"]["normalize_0_1"]["mean"] = 0
pipeline_multi["data_preprocessing"]["train"]["normalize_0_1"]["std"] = 255

pipeline_multi["data_preprocessing"]["train"]["random_rotation"] = {}
pipeline_multi["data_preprocessing"]["train"]["random_rotation"]["name"] =  "random_rotation"
pipeline_multi["data_preprocessing"]["train"]["random_rotation"]["degrees"] = 90
pipeline_multi["data_preprocessing"]["train"]["random_rotation"]["expand"] = True

pipeline_multi["data_preprocessing"]["train"]["random_crop"] = {}
pipeline_multi["data_preprocessing"]["train"]["random_crop"]["name"] = "random_resized_crop"
pipeline_multi["data_preprocessing"]["train"]["random_crop"]["output_shape"] = 224

pipeline_multi["data_preprocessing"]["train"]["horizontal_flip"] = {}
pipeline_multi["data_preprocessing"]["train"]["horizontal_flip"]["name"] = "random_horizontal_flip"

pipeline_multi["data_preprocessing"]["train"]["vertical_flip"] ={}
pipeline_multi["data_preprocessing"]["train"]["vertical_flip"]["name"] = "random_vertical_flip"

pipeline_multi["data_preprocessing"]["train"]["normalize_model_values"] = {}
pipeline_multi["data_preprocessing"]["train"]["normalize_model_values"]["name"] = "normalize"
pipeline_multi["data_preprocessing"]["train"]["normalize_model_values"]["mean"] = [0.485, 0.456, 0.406]
pipeline_multi["data_preprocessing"]["train"]["normalize_model_values"]["std"] = [0.229, 0.224, 0.225]


pipeline_multi["data_preprocessing"]["valid"] = OrderedDict()

pipeline_multi["data_preprocessing"]["valid"]["normalize_0_1"] = {}
pipeline_multi["data_preprocessing"]["valid"]["normalize_0_1"]["name"] = "normalize"
pipeline_multi["data_preprocessing"]["valid"]["normalize_0_1"]["mean"] = 0
pipeline_multi["data_preprocessing"]["valid"]["normalize_0_1"]["std"] = 255

pipeline_multi["data_preprocessing"]["valid"]["center_crop"] = {}
pipeline_multi["data_preprocessing"]["valid"]["center_crop"]["name"] = "center_crop"
pipeline_multi["data_preprocessing"]["valid"]["center_crop"]["output_shape"] = (224,224)

pipeline_multi["data_preprocessing"]["valid"]["normalize_model_values"] = {}
pipeline_multi["data_preprocessing"]["valid"]["normalize_model_values"]["name"] = "normalize"
pipeline_multi["data_preprocessing"]["valid"]["normalize_model_values"]["mean"] = [0.485, 0.456, 0.406]
pipeline_multi["data_preprocessing"]["valid"]["normalize_model_values"]["std"] = [0.229, 0.224, 0.225]



# ---------------------------------set up networks and parameters------------------------------------

pipeline_multi["model"] = OrderedDict()
pipeline_multi["model"]["resnet50"] = {}
pipeline_multi["model"]["resnet50"]["name"] = "resnet50"
pipeline_multi["model"]["resnet50"]["num_output_neurons"] = 4
#pipeline_multi["model"]["mobile_net"]["num_layers_to_train"] = 4

#pipeline_multi["model"]["conv1"] = {}
#pipeline_multi["model"]["conv1"]["name"] = "conv"
#pipeline_multi["model"]["conv1"]["number_of_kernels"] = 32
#pipeline_multi["model"]["conv1"]["kernel_size"] = 7
#pipeline_multi["model"]["lrelu1"] = {}
#pipeline_multi["model"]["lrelu1"]["name"] = "lrelu"
#pipeline_multi["model"]["maxpool1"] = {}
#pipeline_multi["model"]["maxpool1"]["name"] = "max_pool"
#pipeline_multi["model"]["maxpool1"]["kernel_size"] = 2


#pipeline_multi["model"]["conv2"] = {}
#pipeline_multi["model"]["conv2"]["name"] = "conv"
#pipeline_multi["model"]["conv2"]["number_of_kernels"] = 64
#pipeline_multi["model"]["conv2"]["kernel_size"] = 5
#pipeline_multi["model"]["lrelu2"] = {}
#pipeline_multi["model"]["lrelu2"]["name"] = "lrelu"
#pipeline_multi["model"]["maxpool2"] = {}
#pipeline_multi["model"]["maxpool2"]["name"] = "max_pool"
#pipeline_multi["model"]["maxpool2"]["kernel_size"] = 2


#pipeline_multi["model"]["conv3"] = {}
#pipeline_multi["model"]["conv3"]["name"] = "conv"
#pipeline_multi["model"]["conv3"]["number_of_kernels"] = 128
#pipeline_multi["model"]["conv3"]["kernel_size"] = 3
#pipeline_multi["model"]["lrelu3"] = {}
#pipeline_multi["model"]["lrelu3"]["name"] = "lrelu"
#pipeline_multi["model"]["maxpool3"] = {}
#pipeline_multi["model"]["maxpool3"]["name"] = "max_pool"
#pipeline_multi["model"]["maxpool3"]["kernel_size"] = 3

#pipeline_multi["model"]["flatten4"] = {}
#pipeline_multi["model"]["flatten4"]["name"] = "flatten"

#pipeline_multi["model"]["linear5"] = {}
#pipeline_multi["model"]["linear5"]["name"] = "linear"
#pipeline_multi["model"]["linear5"]["neurons"] = 256
#pipeline_multi["model"]["lrelu5"] = {}
#pipeline_multi["model"]["lrelu5"]["name"] = "lrelu"

#pipeline_multi["model"]["linear6"] = {}
#pipeline_multi["model"]["linear6"]["name"] = "linear"
#pipeline_multi["model"]["linear6"]["neurons"] = 128
#pipeline_multi["model"]["lrelu6"] = {}
#pipeline_multi["model"]["lrelu6"]["name"] = "lrelu"

#pipeline_multi["model"]["output"] = {}
#pipeline_multi["model"]["output"]["name"] = "linear"
#pipeline_multi["model"]["output"]["neurons"] = 4

pipeline_multi["optimizer"] = {}
pipeline_multi["optimizer"]["name"] = "adam"

pipeline_multi["loss"] = {}
pipeline_multi["loss"]["type"] = "cross_entropy"
pipeline_multi["loss"]["use_weighted_loss"] = True
pipeline_multi["loss"]["use_single_neuron"] = False



#---------------------------------------------set up evaluation metrics and parameters------------------------


pipeline_multi["metrics"] = {}

# accuracy
pipeline_multi["metrics"]["simple_accuracy"] = {}
pipeline_multi["metrics"]["simple_accuracy"]["name"] = "accuracy"
pipeline_multi["metrics"]["simple_accuracy"]["type"] = "simple"  

pipeline_multi["metrics"]["balanced_accuracy"] = {}
pipeline_multi["metrics"]["balanced_accuracy"]["name"] = "accuracy"
pipeline_multi["metrics"]["balanced_accuracy"]["type"] = "balanced"

# precision
pipeline_multi["metrics"]["precision"] = {}
pipeline_multi["metrics"]["precision"]["name"] = "precision"
pipeline_multi["metrics"]["precision"]["class_result"] = "COVID"

# recall
pipeline_multi["metrics"]["sensitivity"] = {}
pipeline_multi["metrics"]["sensitivity"]["name"] = "sensitivity"
pipeline_multi["metrics"]["sensitivity"]["class_result"]  = "COVID"

# F1 score
pipeline_multi["metrics"]["f1_score"] = {}
pipeline_multi["metrics"]["f1_score"]["name"] = "F1_score" 
pipeline_multi["metrics"]["f1_score"]["class_result"] = "COVID"

# mcc
pipeline_multi["metrics"]["mcc"] = {}
pipeline_multi["metrics"]["mcc"]["name"] ="mcc" 

#------------------------------------------------Create Plots --------------------------
pipeline_multi["plots"] = ["CM", "LC", "MS"]


In [153]:
start =time.time()
CNN_training(pipeline_multi)
end = time.time()
print("Time taken to train the model : ", datetime.timedelta(seconds=end-start))

Using Device: cpu
Unique labels found and their frequencies: 
Label: COVID, %age: 17.0821
Label: Lung_Opacity, %age: 28.4052
Label: Normal, %age: 48.1571
Label: pneumonia, %age: 6.3556
Splitting data using Simple Stratified with 70.0-30.0 ratio
Split the data successfully. Shape: y_train: (1, 11851, 2) y_valid: (1, 5079, 2)

Fold No: 1 Model: 
 Model(
  (seq_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   

## Test data 

In [ ]:
path_to_json = os.path.join(path_to_multiclass_results, "train", "training_pipeline.json")
save_path = os.path.join(path_to_multiclass_results, "test")

start = time.time()
CNN_prediction(path_to_test_data, path_to_json, save_path)
end = time.time()
print("Time taken to generate predicitons on test data: ",  datetime.timedelta(seconds=end-start))

## Noisy data 

In [ ]:
save_path = os.path.join(path_to_multiclass_results, "noisy_test")

start = time.time()
CNN_prediction(path_to_noisy_test_data, path_to_json, save_path)
end = time.time()
print("Time taken to generate predicitons on noisy test data: ", datetime.timedelta(seconds=end-start))

# Create zip file

In [ ]:
raise ValueError("PROCESSING COMPLETED")

In [ ]:
cmd = "tar -zcvf " + path_to_results + ".tar.gz " + path_to_results
os.system(cmd)

# Joel code

In [ ]:
raise ValueError("Entering Joel Code")

In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from skimage import io, transform

from PIL import Image

class2id = test_count = {'Normal': 0, 'COVID': 1, 'pneumonia': 2, 'Lung_Opacity': 3}

id2class = {0: 'Normal', 1: 'COVID', 2: 'pneumonia', 3:'Lung_Opacity'}

class COVIDxDataset(Dataset):
    def __init__(self, txt_frame_file, images_path, transform=None):
        """
        Args:
            txt_frame_file (string): Path to the txt files with labels.
            images_path (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.covidx_frame = pd.read_csv(txt_frame_file, delim_whitespace=True) 
        
        #self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = images_path
        self.transform = transform
        self.class2id = test_count = {'Normal': 0, 'COVID': 1, 'pneumonia': 2, 'Lung_Opacity': 3}
        self.id2class = {0: 'Normal', 1: 'COVID', 2: 'pneumonia', 3:'Lung_Opacity'}
        
        
    def pil_loader(self, path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
        
    
    def __len__(self):
        return len(self.covidx_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_path = os.path.join(self.root_dir,
                                self.covidx_frame.iloc[idx, 1])
        #print(img_name)
        
        image = self.pil_loader(image_path)
        #image = io.imread(img_name)
        
        
        label = self.covidx_frame.iloc[idx, 2]
        #landmarks = np.array([landmarks])
        #landmarks = landmarks.astype('float').reshape(-1, 2)
        #sample = {'image': image, 'label': label}

        if self.transform is not None:
            image = self.transform(image)
            
        sample = (image, self.class2id[label])

        return sample
        

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms, utils
import matplotlib.pyplot as plt
import time
import os
import copy

import math

from torch.utils.data import random_split

In [ ]:
model_name = 'resnet'
batch_size = 32
num_epochs = 10
feature_extract = True

#TODO: get automatically these parameters
num_classes = 4

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs = 25, is_inception= False):
    since = time.time()
    
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs -1))
        
        #Each epoch as a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                #zero the parameter gradients
                optimizer.zero_grad()
                
                #forward
                # track history
                with torch.set_grad_enabled(phase == 'train'):
                    
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                        
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss /len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))
    
    #load the best model
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
# set model parameters
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [ ]:
# Initialize the model for this run
#model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
#print(model_ft)

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# Load Data

# configure paths to the description txt file and the images folder
#train_txt_file = '/kaggle/input/ismdatasetforclassificationofdieases/ism_dataset/raw_data/train_multi.txt'
#train_images_path = '/kaggle/input/ismdatasetforclassificationofdieases/ism_dataset/raw_data/train'

#test_txt_file = '../COVID-Net/test_split_v2.txt'
#test_images_path = '../covid-chestxray-dataset/data/test'

#train_dataset = covidx.COVIDxDataset(train_txt_file, train_images_path)
#test_dataset = covidx.COVIDxDataset(val_txt_file, val_images_path)

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
batch_size = 128
val_size = 150

#training_dataset = COVIDxDataset(train_txt_file, train_images_path)
#val_size =  math.floor(len(training_dataset)*0.3)
#train_size = len(training_dataset) - val_size
#train_data,val_data = random_split(training_dataset,[train_size,val_size])

#image_datasets = {'train': train_data, 'val': val_data}
#image_datasets = {'train': covidx.COVIDxDataset(train_txt_file, train_images_path, data_transforms['train']), 'val': covidx.COVIDxDataset(test_txt_file, test_images_path, data_transforms['val'])}
#image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
#dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'val']}

#print(dataloaders_dict)

In [ ]:

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
# create the optimizer

#model_ft = model_ft.to(device)

#params_to_update = model_ft.parameters()
#print("Params to learn")

#if feature_extract:
#    params_to_update = []
#    for name, param in model_ft.named_parameters():
#        if param.requires_grad == True:
#            params_to_update.append(param)
#            print("\t", name)
#else:
#    for name, param in model_ft.named_parameters():
 #       if param.requires_grad == True:
  #          print("\t", name)
 #           
#optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum = 0.9)
#criterion = nn.CrossEntropyLoss()

#model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs = num_epochs, is_inception=(model_name=="inception"))